<a href="https://colab.research.google.com/github/mohsenfff/Babystore-BEPA/blob/master/Phase1_STEP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -------------------------------
# Install Necessary Packages
# -------------------------------
# Install rarfile library
!pip install rarfile
!pip install rarfile ta ipywidgets tqdm
!apt-get install -y unrar
# Install unrar utility
!apt-get install unrar
!pip install ta ipywidgets tqdm
!pip install --upgrade scikit-learn imbalanced-learn

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.3 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=292be625c176887bd92a75379c8bb11bf2e1c2b2574c0a09a72032b7b1abd2c7
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:

# -------------------------------
# Import Libraries
# -------------------------------
import pandas as pd
import numpy as np
from ta.trend import EMAIndicator, MACD
from ta.volatility import AverageTrueRange
from ta.momentum import RSIIndicator
from datetime import datetime
import pytz
import random
import logging
import sys
import os
from google.colab import files

# Additional imports for GUI and profile handling
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from IPython.display import HTML
from tqdm.notebook import tqdm  # For progress bars
from ipywidgets import GridspecLayout
import zipfile
import rarfile  # For RAR file handling
import io
import shutil  # For directory removal
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
import joblib
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # Catch all FutureWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)  # Catch generic deprecations
# ============================
# Logging Configuration
# ============================
logging.basicConfig(
    level=logging.DEBUG,  # Set to DEBUG to capture all levels of logs
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("trading_bot_debug.log"),
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger()

# ============================
# Global Variables
# ============================
sl_adjustments = {}
signal_distance_adjustments = {}  # Existing variable for Signal Distance settings
rr_adjustments = {}  # New variable for Risk-Reward (RR) settings
selected_asset = None
selected_timeframe = None
uploaded = {}
atr_settings = {}
# Define signal types
signals = ['buy20', 'buy50', 'buy200', 'continue_buy20', 'continue_buy50',
           'sell20', 'sell50', 'sell200', 'continue_sell10', 'continue_sell20', 'continue_sell50']
# ============================
# Helper Functions
# ============================
def save_model_to_drive(model_artifacts, base_path='/content/drive/MyDrive/trading_models/mfe_predictor'):
    """Save model to Google Drive for persistence"""
    from google.colab import drive

    try:
        # Mount Google Drive
        drive.mount('/content/drive')

        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(base_path), exist_ok=True)

        # Save model artifacts
        joblib.dump(model_artifacts['model'], f"{base_path}_model.pkl")
        joblib.dump(model_artifacts['scaler'], f"{base_path}_scaler.pkl")
        joblib.dump(model_artifacts['features'], f"{base_path}_features.pkl")
        joblib.dump(model_artifacts['thresholds'], f"{base_path}_thresholds.pkl")

        logger.info(f"Model artifacts saved to Google Drive: {base_path}")
        return True
    except Exception as e:
        logger.error(f"Error saving model to Drive: {e}")
        return False

def load_model_from_drive(base_path='/content/drive/MyDrive/trading_models/mfe_predictor'):
    """Load model from Google Drive"""
    from google.colab import drive

    try:
        # Mount Google Drive
        drive.mount('/content/drive')

        # Check if model files exist
        if all(os.path.exists(f"{base_path}_{ext}") for ext in ["model.pkl", "scaler.pkl", "features.pkl", "thresholds.pkl"]):
            # Load model artifacts
            model_artifacts = {
                'model': joblib.load(f"{base_path}_model.pkl"),
                'scaler': joblib.load(f"{base_path}_scaler.pkl"),
                'features': joblib.load(f"{base_path}_features.pkl"),
                'thresholds': joblib.load(f"{base_path}_thresholds.pkl")
            }
            logger.info(f"Model artifacts loaded from Google Drive: {base_path}")
            return model_artifacts
        else:
            logger.warning(f"One or more model files not found in Google Drive: {base_path}")
            return None
    except Exception as e:
        logger.error(f"Error loading model from Drive: {e}")
        return None

def extract_csv_from_archive(file_content, filename):
    """
    Extracts CSV files from ZIP or RAR archives.
    Returns a list of tuples containing (extracted_filename, DataFrame).
    """
    extracted_files = []
    if filename.lower().endswith('.zip'):
        try:
            with zipfile.ZipFile(io.BytesIO(file_content)) as z:
                for zip_inner_file in z.namelist():
                    if zip_inner_file.lower().endswith('.csv'):
                        with z.open(zip_inner_file) as csvfile:
                            try:
                                df = pd.read_csv(csvfile)
                                combined_filename = f"{filename}:{os.path.basename(zip_inner_file)}"
                                extracted_files.append((combined_filename, df))
                                logger.debug(f"Successfully extracted CSV from ZIP: {combined_filename}")
                            except Exception as e:
                                logger.error(f"Error reading {zip_inner_file} from ZIP {filename}: {e}")
                    else:
                        logger.debug(f"Skipped non-CSV file in ZIP: {zip_inner_file}")
        except zipfile.BadZipFile:
            logger.error(f"Error: '{filename}' is not a valid ZIP file.")
        except Exception as e:
            logger.error(f"Error processing ZIP file {filename}: {e}")
    elif filename.lower().endswith('.rar'):
        try:
            with rarfile.RarFile(io.BytesIO(file_content)) as r:
                for rar_inner_file in r.namelist():
                    if rar_inner_file.lower().endswith('.csv'):
                        with r.open(rar_inner_file) as csvfile:
                            try:
                                df = pd.read_csv(csvfile)
                                combined_filename = f"{filename}:{os.path.basename(rar_inner_file)}"
                                extracted_files.append((combined_filename, df))
                                logger.debug(f"Successfully extracted CSV from RAR: {combined_filename}")
                            except Exception as e:
                                logger.error(f"Error reading {rar_inner_file} from RAR {filename}: {e}")
                    else:
                        logger.debug(f"Skipped non-CSV file in RAR: {rar_inner_file}")
        except rarfile.BadRarFile:
            logger.error(f"Error: '{filename}' is not a valid RAR file.")
        except rarfile.NeedFirstVolume:
            logger.error(f"Error: '{filename}' is part of a multi-volume RAR archive. Please upload all parts.")
        except Exception as e:
            logger.error(f"Error processing RAR file {filename}: {e}")
    else:
        logger.warning(f"Unsupported archive format for file: {filename}")
    return extracted_files

def load_data():
    """
    Function to upload and load signal and historical data CSV files.
    Accepts CSV, ZIP, and RAR files for both signal and historical data.
    Returns combined DataFrames for signals and historical data.
    """
    # Helper function to handle uploads
    def upload_files(file_description):
        """
        Handles file uploads and extracts CSVs from ZIP/RAR archives.
        Returns a list of tuples (filename, DataFrame).
        """
        print(f"Please upload the {file_description} file(s) (CSV, ZIP, RAR).")
        uploaded_files = files.upload()
        if not uploaded_files:
            logger.warning(f"No files uploaded for {file_description}.")
            return []

        all_files = []
        for filename, file_content in uploaded_files.items():
            logger.debug(f'Uploaded file: {filename}')
            if filename.lower().endswith('.csv'):
                try:
                    df = pd.read_csv(io.BytesIO(file_content))
                    all_files.append((filename, df))
                    logger.debug(f"Successfully loaded CSV file: {filename}")
                except Exception as e:
                    logger.error(f"Error reading CSV file {filename}: {e}")
            elif filename.lower().endswith(('.zip', '.rar')):
                extracted = extract_csv_from_archive(file_content, filename)
                all_files.extend(extracted)
            else:
                logger.warning(f"Unsupported file type: {filename}. Only CSV, ZIP, and RAR files are allowed.")
        return all_files

    # Upload and process Signal files
    signal_files = upload_files("signal data")
    if not signal_files:
        logger.error("No valid signal data files were uploaded.")
        return None, None
    # Combine all signal DataFrames
    signals = pd.concat([df for _, df in signal_files], ignore_index=True)
    logger.info(f"Total Signal records loaded: {len(signals)}")

    # Upload and process Historical data files
    historical_files = upload_files("historical data")
    if not historical_files:
        logger.error("No valid historical data files were uploaded.")
        return signals, None
    # Combine all historical DataFrames
    historical = pd.concat([df for _, df in historical_files], ignore_index=True)
    logger.info(f"Total Historical records loaded: {len(historical)}")

    # Parse Entry_Time and Exit_Time
    if 'Entry_Time' in signals.columns and 'Exit_Time' in signals.columns:
        signals['Entry_Time'] = pd.to_datetime(signals['Entry_Time'], errors='coerce')
        signals['Exit_Time'] = pd.to_datetime(signals['Exit_Time'], errors='coerce')
        logger.info("Parsed 'Entry_Time' and 'Exit_Time' columns in signals data.")
    else:
        logger.error("Error: 'Entry_Time' and/or 'Exit_Time' columns not found in signals data.")
        return None, None

    # Parse historical data time
    if 'time' in historical.columns:
        historical['time'] = pd.to_datetime(historical['time'], errors='coerce')
        historical.set_index('time', inplace=True)
        logger.info("Parsed 'time' column in historical data.")
    else:
        logger.error("Error: 'time' column not found in historical data.")
        return signals, None

    return signals, historical


def calculate_sl_price(signal_name, original_sl_price, signal_type, row=None):
    """
    Calculates the stop-loss price using either:
    1. Percentage-based method (if ATR is disabled or unavailable)
    2. ATR-based method (if enabled and available)

    Parameters:
    signal_name (str): Name of the signal (e.g., 'buy20', 'sell50')
    original_sl_price (float): Original stop-loss price from price action
    signal_type (str): 'buy' or 'sell'
    row (pandas.Series, optional): Current row data with price and ATR

    Returns:
    float: The calculated stop-loss price
    """
    # Check if we should use ATR-based stop-loss
    use_atr = atr_settings.get('use_atr', False)

    # If ATR is enabled and row data is available with ATR
    if use_atr and row is not None and 'ATR' in row and not pd.isna(row['ATR']):
        # Get ATR multiplier for this signal
        atr_multiplier = atr_settings.get(f'{signal_name}_multiplier', 2.5)
        atr_value = row['ATR']
        entry_price = row['close']

        logger.debug(f"Using ATR-based SL for {signal_name}: ATR={atr_value}, Multiplier={atr_multiplier}")

        # Simple ATR-based calculation (inspired by paste.txt approach)
        if signal_type == 'buy':
            # For buy signals, stop-loss is below entry price
            sl_price = entry_price - (atr_multiplier * atr_value)
        else:  # 'sell'
            # For sell signals, stop-loss is above entry price
            sl_price = entry_price + (atr_multiplier * atr_value)
    else:
        # Fallback to percentage-based method
        sl_percentage = sl_adjustments.get(signal_name, 0.0) / 100  # Convert to decimal

        logger.debug(f"Using percentage-based SL for {signal_name}: {sl_percentage*100}%")

        if signal_type == 'buy':
            sl_price = original_sl_price * (1 - sl_percentage)
        else:  # 'sell'
            sl_price = original_sl_price * (1 + sl_percentage)

    return round(sl_price, 5)

def analyze_trailing_stop_performance(trade_log_df):
    """Analyze how trailing stops affected performance"""

    # Count various trade outcomes
    trailing_activated_count = trade_log_df['Trailing_Activated'].sum()
    trailing_sl_hit_count = trade_log_df[trade_log_df['Trade_Result'] == 'Trailing SL Hit'].shape[0]
    normal_sl_hit_count = trade_log_df[trade_log_df['Trade_Result'] == 'SL Hit'].shape[0]
    tp_hit_count = trade_log_df[trade_log_df['Trade_Result'] == 'TP Hit'].shape[0]

    # Calculate profits by outcome
    trailing_sl_profit = trade_log_df[trade_log_df['Trade_Result'] == 'Trailing SL Hit']['Profit'].sum()
    normal_sl_profit = trade_log_df[trade_log_df['Trade_Result'] == 'SL Hit']['Profit'].sum()
    tp_profit = trade_log_df[trade_log_df['Trade_Result'] == 'TP Hit']['Profit'].sum()

    # Calculate effectiveness metrics
    if trailing_activated_count > 0:
        trailing_conversion_rate = trailing_sl_hit_count / trailing_activated_count
    else:
        trailing_conversion_rate = 0

    # Print analysis
    print("\n=== Trailing Stop Performance Analysis ===")
    print(f"Trades with trailing activated: {trailing_activated_count}")
    print(f"Trades exited by trailing stop: {trailing_sl_hit_count} ({trailing_conversion_rate:.1%} conversion)")
    print(f"Trades exited by normal stop: {normal_sl_hit_count}")
    print(f"Trades exited by take profit: {tp_hit_count}")
    print("\nProfit Distribution:")
    print(f"Trailing stop exits: ${trailing_sl_profit:.2f}")
    print(f"Normal stop exits: ${normal_sl_profit:.2f}")
    print(f"Take profit exits: ${tp_profit:.2f}")

    return {
        'trailing_activated': trailing_activated_count,
        'trailing_sl_hit': trailing_sl_hit_count,
        'normal_sl_hit': normal_sl_hit_count,
        'tp_hit': tp_hit_count,
        'trailing_sl_profit': trailing_sl_profit,
        'normal_sl_profit': normal_sl_profit,
        'tp_profit': tp_profit,
        'trailing_conversion_rate': trailing_conversion_rate
    }

def calculate_adjusted_sl_price(signal_name, sl_price, signal_type):
    """
    Calculates the adjusted Stop-Loss price based on the SL percentage.
    """
    sl_percentage = sl_adjustments.get(signal_name, 0.0) / 100  # Convert percentage to decimal
    logger.debug(f"Calculating SL for signal: {signal_name}")
    logger.debug(f"Signal Type: {signal_type}, SL Percentage: {sl_percentage*100}%")

    if signal_type == 'buy':
        adjusted_sl_price = sl_price * (1 - sl_percentage)
    else:  # 'sell'
        adjusted_sl_price = sl_price * (1 + sl_percentage)

    adjusted_sl_price = round(adjusted_sl_price, 5)
    logger.debug(f"Adjusted SL Price: {adjusted_sl_price}")

    return adjusted_sl_price

def calculate_tp_price(signal_name, entry_price, sl_price, signal_type, mfe_range=None):
    """
    Calculates the Take Profit (TP) price based on the RR ratio.
    Supports both fixed and dynamic RR based on MFE range prediction.

    Args:
        signal_name: Name of the signal (e.g. 'sell20', 'buy50')
        entry_price: Entry price of the trade
        sl_price: Stop loss price
        signal_type: Type of trade ('buy' or 'sell')
        mfe_range: Optional ML-predicted MFE range (1=Low, 2=Medium, 3=High)

    Returns:
        Calculated TP price
    """
    # Get base RR ratio from settings
    base_rr = rr_adjustments.get(signal_name, 2.0)

    # Apply dynamic RR adjustment if MFE range is provided
    if mfe_range is not None:
        # Map MFE ranges to multipliers
        rr_mapping = {1: 0.5, 2: 1.0, 3: 1.5}  # Low, Medium, High
        dynamic_factor = rr_mapping.get(mfe_range, 1.0)
        final_rr = base_rr * dynamic_factor
        logger.debug(f"Dynamic TP: signal={signal_name}, base_rr={base_rr}, mfe_range={mfe_range}, final_rr={final_rr}")
    else:
        final_rr = base_rr
        logger.debug(f"Fixed TP: signal={signal_name}, rr={base_rr}")

    # Calculate TP price based on signal type
    if signal_type == 'buy':
        sl_distance = entry_price - sl_price  # Positive value
        tp_distance = sl_distance * final_rr
        tp_price = entry_price + tp_distance
    else:  # 'sell'
        sl_distance = sl_price - entry_price  # Positive value
        tp_distance = sl_distance * final_rr
        tp_price = entry_price - tp_distance

    tp_price = round(tp_price, 5)
    logger.debug(f"Calculated TP Price: {tp_price}")

    return tp_price

def check_model_exists(model_path='models/mfe_predictor'):
    """Check if the trained model files exist"""
    required_files = [
        f"{model_path}_model.pkl",
        f"{model_path}_scaler.pkl",
        f"{model_path}_features.pkl",
        f"{model_path}_thresholds.pkl"
    ]
    return all(os.path.exists(f) for f in required_files)

def load_mfe_model(model_path='models/mfe_predictor'):
    """Load the trained MFE prediction model"""
    try:
        return {
            'model': joblib.load(f"{model_path}_model.pkl"),
            'scaler': joblib.load(f"{model_path}_scaler.pkl"),
            'features': joblib.load(f"{model_path}_features.pkl"),
            'thresholds': joblib.load(f"{model_path}_thresholds.pkl")
        }
    except Exception as e:
        logger.error(f"Error loading MFE model: {e}")
        return None

def predict_mfe_range(row, model_artifacts):
    """Make MFE range prediction with explicit handling for Trade_Type_Encoded"""
    try:
        # Extract features needed by the model
        features = model_artifacts['features']

        # Create a DataFrame with this row's data
        X = pd.DataFrame([row.copy()])

        # Build feature DataFrame with exact matching columns
        X_features = pd.DataFrame()

        # CRITICAL: Explicitly handle the Trade_Type_Encoded feature
        for feature in features:
            if feature == 'Trade_Type_Encoded':
                # Determine if it's a buy or sell signal
                signal_type = str(row.get('signal', '')).lower()
                is_buy = any(buy_term in signal_type for buy_term in ['buy', 'long'])
                X_features['Trade_Type_Encoded'] = [1 if is_buy else -1]
            elif feature in X.columns:
                X_features[feature] = X[feature]
            else:
                X_features[feature] = 0

        # Print debugging info - remove in production
        print(f"Features for prediction: {X_features.columns.tolist()}")

        # Scale features
        X_scaled = model_artifacts['scaler'].transform(X_features)

        # Make prediction
        mfe_range = int(model_artifacts['model'].predict(X_scaled)[0])
        return mfe_range
    except Exception as e:
        logger.error(f"MFE prediction failed: {e}")
        return None

def check_wl_model_exists(model_path='models/win_loss_predictor'):
    """Check if the trained Win/Loss model files exist"""
    required_files = [
        f"{model_path}_model.pkl",
        f"{model_path}_scaler.pkl",
        f"{model_path}_features.pkl",
        f"{model_path}_thresholds.pkl" # Keep for consistency, even if not strictly needed
    ]
    return all(os.path.exists(f) for f in required_files)

def load_wl_model(model_path='models/win_loss_predictor'):
    """Load the trained Win/Loss prediction model"""
    try:
        return {
            'model': joblib.load(f"{model_path}_model.pkl"),
            'scaler': joblib.load(f"{model_path}_scaler.pkl"),
            'features': joblib.load(f"{model_path}_features.pkl"),
            'thresholds': joblib.load(f"{model_path}_thresholds.pkl")
        }
    except Exception as e:
        logger.error(f"Error loading Win/Loss model: {e}")
        return None
# -------------------------------
# Performance Metrics Helper Functions
# -------------------------------
def calculate_max_drawdown(equity_curve):
    """
    Calculate the maximum drawdown (%) given an equity curve (cumulative profit Series).
    """
    running_max = equity_curve.cummax()
    drawdown = (equity_curve - running_max) / running_max * 100
    return drawdown.min()  # This will be negative (the worst drawdown)

def calculate_longest_streak(condition_series):
    """
    Calculate the longest consecutive True values in a boolean Series.
    """
    longest = current = 0
    for val in condition_series:
        if val:
            current += 1
            if current > longest:
                longest = current
        else:
            current = 0
    return longest

def calculate_drawdown_series(equity_curve):
    """
    Calculate the drawdown series (%) for an equity curve.
    """
    running_max = equity_curve.cummax()
    drawdown = (equity_curve - running_max) / running_max * 100
    return drawdown

def compute_performance_metrics(df):
    """
    Compute key performance metrics for a trade log DataFrame.
    Assumes df has at least the following columns: Profit.
    Returns a dictionary of metrics.
    """
    total_profit = df['Profit'].sum()
    total_trades = len(df)
    wins = df[df['Profit'] > 0]
    losses = df[df['Profit'] < 0]
    win_rate = (len(wins) / total_trades * 100) if total_trades > 0 else 0
    profit_factor = (wins['Profit'].sum() / abs(losses['Profit'].sum())
                     if losses['Profit'].sum() != 0 else float('inf'))

    equity_curve = df['Profit'].cumsum()
    max_drawdown = calculate_max_drawdown(equity_curve)

    avg_profit = total_profit / total_trades if total_trades > 0 else 0
    sharpe_ratio = (df['Profit'].mean() / df['Profit'].std()
                    if df['Profit'].std() != 0 else float('nan'))

    longest_win_streak = calculate_longest_streak(df['Profit'] > 0)
    longest_loss_streak = calculate_longest_streak(df['Profit'] < 0)

    return {
        "Total Profit": total_profit,
        "Win Rate (%)": win_rate,
        "Profit Factor": profit_factor,
        "Max Drawdown (%)": max_drawdown,
        "Sharpe Ratio": sharpe_ratio,
        "Average Profit per Trade": avg_profit,
        "Longest Winning Streak": longest_win_streak,
        "Longest Losing Streak": longest_loss_streak
    }

def predict_win_probability(row, model_artifacts):
    """Make Win/Loss probability prediction (confidence score)"""
    try:
        # Extract features needed by the model
        features = model_artifacts['features']

        # Create a DataFrame with this row's data
        X = pd.DataFrame([row.copy()])

        # Build feature DataFrame with exact matching columns
        X_features = pd.DataFrame()

        # CRITICAL: Explicitly handle the Trade_Type_Encoded feature
        for feature in features:
            if feature == 'Trade_Type_Encoded':
                # Determine if it's a buy or sell signal
                signal_type = str(row.get('signal', '')).lower()
                is_buy = any(buy_term in signal_type for buy_term in ['buy', 'long'])
                X_features['Trade_Type_Encoded'] = [1 if is_buy else -1]
            elif feature in X.columns:
                X_features[feature] = X[feature]
            else:
                X_features[feature] = 0 # Default to 0 for missing features

        # Scale features
        X_scaled = model_artifacts['scaler'].transform(X_features)

        # Make prediction - assuming model.predict_proba outputs probabilities
        # Assuming binary classification (Win/Loss), probability of winning is often the probability of class '1'
        win_probability = model_artifacts['model'].predict_proba(X_scaled)[0][1]

        logger.debug(f"Win probability for signal {row.get('signal', 'unknown')}: {win_probability:.4f}")
        return win_probability

    except Exception as e:
        logger.error(f"Win/Loss prediction failed: {e}")
        return 0.5 # Default to 0.5 (no edge) in case of error

def calculate_dynamic_position_size(ml_confidence_score, portfolio_value, drawdown_status=None,
                                    fixed_predicted_rr=1.5, base_kelly_fraction=0.2,
                                    max_position_size_percentage=0.05):
    """
    Calculates dynamic position size using Fractional Kelly, scaled by ML confidence.

    Args:
        ml_confidence_score (float): Probability of a winning trade from ML model (0-1).
        portfolio_value (float): Current portfolio/equity value.
        drawdown_status (dict): Current drawdown metrics (optional).
        fixed_predicted_rr (float): Estimated RR for this trade. Default=1.5.
        base_kelly_fraction (float): Fraction of Kelly. 0.2 => 20%.
        max_position_size_percentage (float): Maximum position size as % of portfolio.

    Returns:
        float: Position size in monetary terms (units of currency).
    """
    # 1) Calculate edge
    edge = (ml_confidence_score * fixed_predicted_rr) - (1.0 - ml_confidence_score)

    # 2) If edge <= 0, no positive expectation => zero position
    if edge <= 0:
        return 0.0

    # 3) Apply drawdown scaling if provided
    kelly_scaler = 1.0
    if drawdown_status:
        if drawdown_status.get('total_drawdown_pct', 0) >= 0.10:
            kelly_scaler = 0.2  # Reduce to 20% on heavy drawdown
        elif drawdown_status.get('daily_drawdown_pct', 0) >= 0.05:
            kelly_scaler = 0.5  # Reduce to 50% on moderate drawdown

    # 4) Calculate effective Kelly fraction
    effective_kelly_fraction = base_kelly_fraction * kelly_scaler

    # 5) Calculate raw Kelly-based size
    position_size = effective_kelly_fraction * (edge / fixed_predicted_rr) * portfolio_value

    # 6) Enforce max position size constraint
    max_position_size = max_position_size_percentage * portfolio_value
    position_size = min(position_size, max_position_size)

    # 7) Final check for non-negative position
    return max(position_size, 0.0)
# -------------------------------
#Profile managment UI and Functions
# -------------------------------
def profile_management_menu():
    clear_output()

    print("### Trading Strategy Backtest with Dynamic Stop-Loss, Signal Distance, and Risk-Reward Adjustment ###\n")

    options = ['Create a new profile', 'Load a profile', 'Update an existing profile']
    menu = widgets.RadioButtons(options=options, description='Options:', disabled=False)

    next_button = widgets.Button(description='Next', button_style='success')
    back_button = widgets.Button(description='Back', button_style='warning')
    back_button.layout.visibility = 'hidden'  # Hide back button on main menu

    display(menu)
    display(widgets.HBox([next_button, back_button]))

    def on_next_button_clicked(b):
        selection = menu.value
        if selection == 'Create a new profile':
            create_new_profile()
        elif selection == 'Load a profile':
            load_profile()
        elif selection == 'Update an existing profile':
            update_existing_profile()
        else:
            print("Please select an option before proceeding.")

    next_button.on_click(on_next_button_clicked)

    # Add a back button handler to return to the new main menu
    def on_back_to_main_clicked(b):
        main_menu()

    back_button.on_click(on_back_to_main_clicked)
    back_button.layout.visibility = 'visible'  # Show back button
# -------------------------------
# Train Model UI and Functions
# -------------------------------
def train_model_config_ui():
    clear_output()
    print("### Configure Model Training ###\n")

    # Create an output widget to capture debug prints
    out = widgets.Output()

    # Configuration widgets
    test_size_slider = widgets.FloatSlider(
        value=0.2,
        min=0.1,
        max=0.5,
        step=0.05,
        description='Test Size:',
        readout_format='.2f'
    )
    n_splits_slider = widgets.IntSlider(
        value=5,
        min=3,
        max=10,
        step=1,
        description='TimeSeriesSplits:'
    )
    quantile_checkbox = widgets.Checkbox(
        value=False,
        description='Use Quantile Thresholds'
    )
    model_save_path_text = widgets.Text(
        value='models/mfe_predictor',
        description='Model Save Path:'
    )

    # Button to trigger file upload using Colab's default uploader
    upload_button = widgets.Button(
        description="Upload Trade Log CSV",
        button_style='info'
    )
    train_button = widgets.Button(
        description='Train Model',
        button_style='success'
    )
    back_button = widgets.Button(
        description='Back',
        button_style='warning'
    )

    # Layout the UI elements
    config_ui = widgets.VBox([
        test_size_slider,
        n_splits_slider,
        quantile_checkbox,
        model_save_path_text,
        upload_button,
        widgets.HBox([train_button, back_button]),
        out  # Output widget placed at the bottom
    ])
    display(config_ui)

    # Dictionary to store uploaded file details
    uploaded_file = {}

    def on_upload_button_clicked(b):
        from google.colab import files as gfiles
        with out:
            out.clear_output()
            print("Opening file dialog...")
        uploaded = gfiles.upload()  # Opens Colab's file dialog
        if not uploaded:
            with out:
                print("No file uploaded.")
        else:
            key = list(uploaded.keys())[0]  # Expect only one file
            uploaded_file["filename"] = key
            uploaded_file["content"] = uploaded[key]
            with out:
                print(f"File '{key}' uploaded successfully.")

    upload_button.on_click(on_upload_button_clicked)

    def on_train_button_clicked(b):
        with out:
            print("Train button clicked.")
            print("Uploaded file dictionary contents:", uploaded_file)
            if not uploaded_file:
                print("Please upload a trade log CSV file before training.")
                return
            import io
            print("Attempting to read the CSV file...")
            try:
                trade_log_df = pd.read_csv(io.BytesIO(uploaded_file["content"]))
                print(f"Trade log CSV '{uploaded_file['filename']}' loaded with {len(trade_log_df)} records.")
            except Exception as e:
                print("Error loading CSV file:", e)
                return
            print("Starting model training... This may take a while.")
            artifacts = build_rr_prediction_model(
                trade_log_df,
                use_quantile_thresholds=quantile_checkbox.value,
                model_save_path=model_save_path_text.value,
                n_splits=n_splits_slider.value
            )
            print("Model training complete.")
            main_menu()

    train_button.on_click(on_train_button_clicked)

    def on_back_button_clicked(b):
        main_menu()

    back_button.on_click(on_back_button_clicked)

# -------------------------------
# Predict & Generate UI and Functions
# -------------------------------

def predict_generate_ui():
    clear_output()
    print("### Predict & Generate (Dynamic TP) ###\n")

    # First, we need to show the profile selection
    print("First, please select a trading profile to set SL/TP parameters.\n")

    # Create a container for the different stages of the UI
    main_container = widgets.VBox([])
    display(main_container)

    # Define profile selection UI
    def show_profile_selection():
        profile_dir = 'profiles'
        if not os.path.exists(profile_dir):
            print("No profiles found. Please create a profile first in the 'Generate Signal & Backtest (Classic)' menu.")
            return False

        profile_files = [f for f in os.listdir(profile_dir) if f.endswith('.json')]
        if not profile_files:
            print("No profiles found. Please create a profile first.")
            return False

        profile_dropdown = widgets.Dropdown(
            options=profile_files,
            description='Select Profile:',
            disabled=False
        )

        load_button = widgets.Button(
            description='Load Profile',
            button_style='success'
        )

        back_button = widgets.Button(
            description='Back',
            button_style='warning'
        )

        # Update the main container
        main_container.children = [
            widgets.VBox([
                profile_dropdown,
                widgets.HBox([load_button, back_button])
            ])
        ]

        def on_load_button_clicked(b):
            profile_name = profile_dropdown.value
            if not profile_name:
                print("Please select a profile to load.")
                return

            profile_path = os.path.join('profiles', profile_name)
            with open(profile_path, 'r') as f:
                profile_data = json.load(f)

            global sl_adjustments, signal_distance_adjustments, rr_adjustments, atr_settings, selected_asset, selected_timeframe
            sl_adjustments = profile_data['sl_adjustments']
            signal_distance_adjustments = profile_data['signal_distance_adjustments']
            rr_adjustments = profile_data['rr_adjustments']
            atr_settings = profile_data.get('atr_settings', {})  # Get ATR settings, default to empty dict
            selected_asset = profile_data['asset']
            selected_timeframe = profile_data['timeframe']

            # Log loaded ATR settings
            logger.debug("Loaded ATR Settings:")
            logger.debug(f"Use ATR: {atr_settings.get('use_atr', False)}")
            logger.debug(f"ATR Period: {atr_settings.get('period', 14)}")
            logger.debug(f"Use Trailing Stop: {atr_settings.get('use_trailing_stop', False)}")
            logger.debug(f"Activation %: {atr_settings.get('activation_pct', 0.5)}")

            for signal in signals:
                mult = atr_settings.get(f'{signal}_multiplier', 1.8)
                logger.debug(f"Signal: {signal}, ATR Multiplier: {mult}")
            print(f"\nProfile '{profile_name}' loaded successfully.")
            print(f"Selected asset: {selected_asset}")
            print(f"Selected timeframe: {selected_timeframe}")

            # Proceed to model selection and file upload
            show_model_selection()

        def on_back_button_clicked(b):
            main_menu()

        load_button.on_click(on_load_button_clicked)
        back_button.on_click(on_back_button_clicked)
        return True

    # Define model selection and file upload UI
    def show_model_selection():
        # Create an output widget for messages
        out = widgets.Output()

        # UI widgets
        model_path_text = widgets.Text(
            value='models/mfe_predictor',
            description='Model Path:',
            style={'description_width': 'initial'}
        )

        wl_model_path_text = widgets.Text(
            value='models/wl_predictor',
            description='WL Model Path:',
            style={'description_width': 'initial'}
        )

        dynamic_ps_checkbox = widgets.Checkbox(
            value=True,
            description='Use Dynamic PS'
        )

        backtest_checkbox = widgets.Checkbox(
            value=True,
            description='Run Backtesting'
        )

        upload_button = widgets.Button(
            description="Upload Historical Data",
            button_style='info'
        )

        predict_button = widgets.Button(
            description='Generate Signals',
            button_style='success',
            disabled=True  # Disabled until file is uploaded
        )

        back_button = widgets.Button(
            description='Back',
            button_style='warning'
        )

        # Store uploaded file
        uploaded_file = {}

        mfe_model_path_text = widgets.Text(
            value='models/mfe_predictor',
            description='RR Model Path:',
            style={'description_width': 'initial'}
        )


        # Update the main container
        main_container.children = [
            widgets.VBox([
                widgets.HTML(value="<b>Step 2: Select ML Model and Upload Data</b>"),
                model_path_text,
                mfe_model_path_text,
                wl_model_path_text,
                dynamic_ps_checkbox,
                backtest_checkbox,
                upload_button,
                widgets.HBox([predict_button, back_button]),
                out
            ])
        ]

        def on_upload_button_clicked(b):
            from google.colab import files as gfiles
            with out:
                out.clear_output()
                print("Opening file dialog for historical data...")
            uploaded = gfiles.upload()
            if not uploaded:
                with out:
                    print("No file uploaded.")
            else:
                key = list(uploaded.keys())[0]
                uploaded_file["filename"] = key
                uploaded_file["content"] = uploaded[key]
                with out:
                    print(f"Historical data file '{key}' uploaded successfully.")
                predict_button.disabled = False  # Enable generate button

        def on_predict_button_clicked(b):
            with out:
                out.clear_output()
                if not uploaded_file:
                    print("Please upload a historical data CSV file first.")
                    return

                filename = uploaded_file["filename"]
                with open(filename, "wb") as f:
                    f.write(uploaded_file["content"])
                print(f"File saved as '{filename}'.")

                # Step 1: Process file to generate signals using the loaded profile
                print(f"Processing file to generate signals using profile for {selected_asset} {selected_timeframe}...")
                print("Using SL adjustments:", sl_adjustments)
                process_file(filename)

                # Step 2: Load the generated signals
                try:
                    # Try to load combined signals with base filename first
                    base_filename = os.path.splitext(filename)[0]
                    signal_file = f"combined_signals_{base_filename}.csv"

                    if os.path.exists(signal_file):
                        signals_df = pd.read_csv(signal_file)
                    else:
                        # Fallback to default name
                        signals_df = pd.read_csv("combined_signals.csv")
                        signal_file = "combined_signals.csv"

                    print(f"Loaded {len(signals_df)} signals from {signal_file}")

                    # Step 3: Load ML model
                    model_artifacts = load_mfe_model(model_path_text.value)
                    if model_artifacts is None:
                        print(f"Error: Could not load model from path: {model_path_text.value}")
                        return
                    print("Model loaded successfully!")

                    # Step 4: Apply ML model to adjust TP values
                    print("Applying ML model to adjust TP values...")
                    dynamic_tp_count = 0

                    for idx, row in signals_df.iterrows():
                        if pd.notna(row.get('signal', '')):
                            signal_type = str(row['signal']).lower()

                            # Skip if we don't have both close and SL_Price
                            if pd.isna(row.get('close', np.nan)) or pd.isna(row.get('SL_Price', np.nan)):
                                continue

                            # Skip if entry price equals stop loss price (would cause issues)
                            if abs(row['close'] - row['SL_Price']) < 0.00001:
                                continue

                            is_buy = any(buy_term in signal_type for buy_term in ['buy', 'long'])
                            trade_type = 'buy' if is_buy else 'sell'

                            try:
                                # Predict MFE range
                                mfe_range = predict_mfe_range(row, model_artifacts)

                                # Calculate dynamic TP
                                tp_price = calculate_tp_price(
                                    signal_type,  # Use actual signal name
                                    row['close'],
                                    row['SL_Price'],
                                    trade_type,
                                    mfe_range=mfe_range
                                )

                                # Update TP value
                                signals_df.at[idx, 'TP_Price'] = tp_price
                                dynamic_tp_count += 1
                            except Exception as e:
                                print(f"Error processing signal at row {idx}: {e}")

                    # Step 5: Save signals with dynamic TP
                    # Use a more descriptive filename
                    base_filename = os.path.splitext(filename)[0]
                    dynamic_signal_file = f"combined_signals_dynamic_{base_filename}.csv"
                    signals_df.to_csv(dynamic_signal_file, index=False)
                    print(f"Successfully updated {dynamic_tp_count} TP values.")
                    print(f"Signals with dynamic TP saved as '{dynamic_signal_file}'")

                    # Step 6: Run backtesting if enabled
                    if backtest_checkbox.value:
                        print("\nRunning backtesting on dynamic signals...")
                        # Call the backtesting function with the dynamic signals file
                        process_file_with_progress(dynamic_signal_file)

                        # Rename the trade log for clarity if needed
                        trade_log_file = f"trade_log_{base_filename}.csv"
                        dynamic_trade_log = f"trade_log_dynamic_{base_filename}.csv"

                        if os.path.exists(trade_log_file):
                            # If we need to rename
                            import shutil
                            shutil.copy(trade_log_file, dynamic_trade_log)
                            print(f"Trade log saved as '{dynamic_trade_log}'")
                    else:
                        print("Backtesting skipped.")

                except Exception as e:
                    import traceback
                    print(f"Error: {e}")
                    traceback.print_exc()

        def on_back_button_clicked(b):
            # Go back to profile selection
            show_profile_selection()

        upload_button.on_click(on_upload_button_clicked)
        predict_button.on_click(on_predict_button_clicked)
        back_button.on_click(on_back_button_clicked)

    # Start with profile selection
    if not show_profile_selection():
        # If no profiles available, provide a back button
        back_button = widgets.Button(
            description='Back to Menu',
            button_style='warning'
        )

        def on_back_clicked(b):
            main_menu()

        back_button.on_click(on_back_clicked)
        main_container.children = [back_button]


# -------------------------------
# Benchmark & Compare UI and Functions
# -------------------------------
def benchmark_compare_ui():
    clear_output()
    print("### Benchmark & Compare Trading Strategies ###\n")

    # Create an output widget to capture debug and display messages.
    out = widgets.Output()

    # Create buttons to upload each trade log using Google Colab's default uploader.
    classic_upload_button = widgets.Button(
        description="Upload Classic Trade Log",
        button_style='info'
    )
    dynamic_upload_button = widgets.Button(
        description="Upload Dynamic Trade Log",
        button_style='info'
    )
    compare_button = widgets.Button(
        description="Compare Strategies",
        button_style='success'
    )
    back_button = widgets.Button(
        description="Back",
        button_style='warning'
    )

    # Layout the UI elements.
    ui = widgets.VBox([
        classic_upload_button,
        dynamic_upload_button,
        widgets.HBox([compare_button, back_button]),
        out
    ])
    display(ui)

    # Dictionaries to store uploaded file details.
    uploaded_classic = {}
    uploaded_dynamic = {}

    def on_classic_upload(b):
        from google.colab import files as gfiles
        with out:
            out.clear_output()
            print("Opening file dialog for Classic Trade Log...")
        uploaded = gfiles.upload()
        if not uploaded:
            with out:
                print("No file uploaded for Classic Trade Log.")
        else:
            key = list(uploaded.keys())[0]
            uploaded_classic["filename"] = key
            uploaded_classic["content"] = uploaded[key]
            with out:
                print(f"Classic Trade Log file '{key}' uploaded successfully.")

    classic_upload_button.on_click(on_classic_upload)

    def on_dynamic_upload(b):
        from google.colab import files as gfiles
        with out:
            out.clear_output()
            print("Opening file dialog for Dynamic Trade Log...")
        uploaded = gfiles.upload()
        if not uploaded:
            with out:
                print("No file uploaded for Dynamic Trade Log.")
        else:
            key = list(uploaded.keys())[0]
            uploaded_dynamic["filename"] = key
            uploaded_dynamic["content"] = uploaded[key]
            with out:
                print(f"Dynamic Trade Log file '{key}' uploaded successfully.")

    dynamic_upload_button.on_click(on_dynamic_upload)

    def on_compare_clicked(b):
        with out:
            out.clear_output()
            print("Comparing strategies...")
            # Ensure both files are uploaded.
            if not uploaded_classic or not uploaded_dynamic:
                print("Please upload both trade log CSV files.")
                return
            import io
            try:
                classic_df = pd.read_csv(io.BytesIO(uploaded_classic["content"]))
                dynamic_df = pd.read_csv(io.BytesIO(uploaded_dynamic["content"]))
            except Exception as e:
                print("Error reading CSV files:", e)
                return

            # Calculate performance metrics for each strategy.
            classic_metrics = compute_performance_metrics(classic_df)
            dynamic_metrics = compute_performance_metrics(dynamic_df)

            # Create a side-by-side comparison table.
            comparison_df = pd.DataFrame({
                "Classic": classic_metrics,
                "Dynamic": dynamic_metrics
            })
            print("Performance Metrics Comparison:")
            display(comparison_df)

            # Visualization 1: Equity Curves
            classic_equity = classic_df['Profit'].cumsum()
            dynamic_equity = dynamic_df['Profit'].cumsum()
            plt.figure(figsize=(10,6))
            plt.plot(classic_equity, label="Classic Equity Curve")
            plt.plot(dynamic_equity, label="Dynamic Equity Curve")
            plt.title("Equity Curves Comparison")
            plt.xlabel("Trade Number")
            plt.ylabel("Cumulative Profit")
            plt.legend()
            plt.show()

            # Visualization 2: Drawdown Comparison
            classic_drawdown = calculate_drawdown_series(classic_equity)
            dynamic_drawdown = calculate_drawdown_series(dynamic_equity)
            plt.figure(figsize=(10,6))
            plt.plot(classic_drawdown, label="Classic Drawdown")
            plt.plot(dynamic_drawdown, label="Dynamic Drawdown")
            plt.title("Drawdown Comparison")
            plt.xlabel("Trade Number")
            plt.ylabel("Drawdown (%)")
            plt.legend()
            plt.show()

            # Visualization 3: Win/Loss Distribution Comparison
            plt.figure(figsize=(10,6))
            plt.hist(classic_df['Profit'], bins=30, alpha=0.5, label="Classic Profit Distribution")
            plt.hist(dynamic_df['Profit'], bins=30, alpha=0.5, label="Dynamic Profit Distribution")
            plt.title("Win/Loss Distribution Comparison")
            plt.xlabel("Profit")
            plt.ylabel("Frequency")
            plt.legend()
            plt.show()

    compare_button.on_click(on_compare_clicked)

    def on_back_clicked(b):
        main_menu()

    back_button.on_click(on_back_clicked)
# ============================
# New Main Menu Interface
# ============================
def main_menu():
    clear_output()

    # Create styled header
    header = widgets.HTML(
        value="<h1 style='color: #2e86c1; text-align: center;'>Advanced Trading Bot</h1>"
        "<h3 style='color: #138d75; text-align: center;'>ML-Driven Risk Management System</h3>"
    )

    # Create menu buttons with descriptions
    menu_items = [
        {
            'title': 'Generate Signal & Backtest (Classic)',
            'desc': 'Traditional strategy with fixed risk parameters',
            'color': '#3498db'
        },
        {
            'title': 'Train & Create Models',
            'desc': 'Train new ML models for RR prediction',
            'color': '#2ecc71'
        },
        {
            'title': 'Predict & Generate (Dynamic TP)',
            'desc': 'Generate signals with ML-driven dynamic take-profit',
            'color': '#e67e22'
        },
        {
            'title': 'Benchmark & Compare',
            'desc': 'Compare different strategy performances',
            'color': '#9b59b6'
        }
    ]

    # Create grid layout
    grid = GridspecLayout(4, 1, width='100%')

    for i, item in enumerate(menu_items):
        # Create button
        btn = widgets.Button(
            description=item['title'],
            layout=widgets.Layout(width='90%', height='50px'),
            button_style='primary',
            style={'button_color': item['color']}
        )

        # Create description
        desc = widgets.HTML(
            value=f"<div style='color: #7f8c8d; font-size: 0.9em; padding-left: 20px;'>{item['desc']}</div>"
        )

        # Create vertical box container
        container = widgets.VBox(
            [btn, desc],
            layout=widgets.Layout(
                border='2px solid ' + item['color'],
                padding='10px',
                margin='5px',
                width='95%'
            )
        )

        # Add to grid
        grid[i, 0] = container

    # Create back button
    back_btn = widgets.Button(
        description='Exit',
        button_style='danger',
        icon='power-off',
        layout=widgets.Layout(width='150px')
    )

    # Define button click handlers
    def on_classic_click(b):
        clear_output()
        print("Launching Classic Mode...")
        profile_management_menu()

    def on_train_click(b):
        clear_output()
        print("Launching Model Training...")
        train_model_config_ui()

    def on_predict_click(b):
        clear_output()
        print("Launching Dynamic TP Generation...")
        predict_generate_ui()

    def on_benchmark_click(b):
        clear_output()
        print("Launching Benchmark Tool...")
        # Placeholder - connect to comparison workflow
        benchmark_compare_ui()

    def on_back_click(b):
        clear_output()
        print("System shutdown...")
        sys.exit()

    # Assign handlers
    grid.children[0].children[0].on_click(on_classic_click)
    grid.children[1].children[0].on_click(on_train_click)
    grid.children[2].children[0].on_click(on_predict_click)
    grid.children[3].children[0].on_click(on_benchmark_click)
    back_btn.on_click(on_back_click)

    # Display all elements
    display(header)
    display(grid)
    display(widgets.HBox([back_btn], layout=widgets.Layout(justify_content='center')))

# Helper function for placeholder features
def not_implemented():
    print("Feature not implemented yet!")
    input("Press Enter to return to main menu...")
    main_menu()

# ============================
# Start the new main menu
# ============================
main_menu()

# ============================
# Create New Profile
# ============================

def create_new_profile():
    clear_output()
    global selected_asset, selected_timeframe
    asset_options = ['EURUSD', 'XAUUSD', 'XTIUSD', 'SP500', 'NAS100', 'US30', 'DAX40', 'BTCUSDT', 'ETHEREUM']
    timeframe_options = ['2 Minutes', '5 Minutes', '15 Minutes', '1 Hour', '4 Hours', '1 Day']

    asset_dropdown = widgets.Dropdown(options=asset_options, description='Select Asset:', disabled=False)
    timeframe_dropdown = widgets.Dropdown(options=timeframe_options, description='Select Timeframe:', disabled=False)
    proceed_button = widgets.Button(description='Proceed', button_style='success')
    back_button = widgets.Button(description='Back', button_style='warning')

    display(asset_dropdown)
    display(timeframe_dropdown)
    display(widgets.HBox([proceed_button, back_button]))

    def on_proceed_button_clicked(b):
        selected_asset = asset_dropdown.value
        selected_timeframe = timeframe_dropdown.value
        if selected_asset and selected_timeframe:
            adjust_sl_sd_rr(asset_dropdown.value, timeframe_dropdown.value)
        else:
            print("Please select both asset and timeframe.")

    def on_back_button_clicked(b):
        main_menu()

    proceed_button.on_click(on_proceed_button_clicked)
    back_button.on_click(on_back_button_clicked)

# ============================
# Adjust Stop-Loss, Signal Distance, and RR
# ============================

def adjust_sl_sd_rr(asset, timeframe, existing_adjustments=None):
    clear_output()
    if existing_adjustments:
        print(f"### Update SL, Signal Distance, and RR Levels for {asset} on {timeframe} ###\n")
    else:
        print(f"### Adjust SL, Signal Distance, and RR Levels for {asset} on {timeframe} ###\n")

    # Define buy and sell signals, including buy200 and sell200
    buy_signals = ['buy20', 'buy50', 'buy200', 'continue_buy20', 'continue_buy50']
    sell_signals = ['sell20', 'sell50', 'sell200', 'continue_sell10', 'continue_sell20', 'continue_sell50']
    signals = buy_signals + sell_signals

    global sl_adjustments, signal_distance_adjustments, rr_adjustments, atr_settings
    if existing_adjustments:
        sl_adjustments = existing_adjustments.get('sl_adjustments', {})
        signal_distance_adjustments = existing_adjustments.get('signal_distance_adjustments', {})
        rr_adjustments = existing_adjustments.get('rr_adjustments', {})
        atr_settings = existing_adjustments.get('atr_settings', {})
    else:
        sl_adjustments = {}
        signal_distance_adjustments = {}
        rr_adjustments = {}
        atr_settings = {}


    inputs_sl = {}
    inputs_sd = {}
    inputs_rr = {}

    num_columns = 3
    num_rows = int(np.ceil(len(signals) / num_columns))

    # ================================
    # === ATR Settings Section ===
    # ================================
    print("\n### ATR-based Stop-Loss Settings ###\n")

    # Create ATR settings container
    atr_container = widgets.VBox()

    # Main ATR settings
    use_atr_checkbox = widgets.Checkbox(
        value=atr_settings.get('use_atr', False),
        description='Use ATR for Stop-Loss',
        indent=False
    )

    atr_period_slider = widgets.IntSlider(
        value=atr_settings.get('period', 14),
        min=5,
        max=50,
        step=1,
        description='ATR Period:',
        disabled=not use_atr_checkbox.value,
        style={'description_width': 'initial'}
    )

    # Trailing stop settings
    use_trailing_stop_checkbox = widgets.Checkbox(
        value=atr_settings.get('use_trailing_stop', False),
        description='Use Trailing Stop',
        indent=False,
        disabled=not use_atr_checkbox.value
    )

    activation_slider = widgets.FloatSlider(
        value=atr_settings.get('activation_pct', 0.5),
        min=0.0,
        max=1.0,
        step=0.05,
        description='Activation %:',
        readout_format='.0%',
        disabled=not use_trailing_stop_checkbox.value,
        style={'description_width': 'initial'},
        tooltip='How far price must move toward TP before trailing activates'
    )

    # ATR multiplier settings
    print("\n### ATR Multipliers by Signal ###\n")

    # Create grid layout for multipliers
    atr_multiplier_container = widgets.VBox()
    atr_multiplier_container.layout.display = 'flex' if use_atr_checkbox.value else 'none'

    multiplier_grid = GridspecLayout(num_rows, num_columns, width='100%')
    atr_multiplier_inputs = {}

    for idx, signal in enumerate(signals):
        row = idx // num_columns
        col = idx % num_columns

        # Color based on signal type
        theme = '#d4fcd4' if signal in buy_signals else '#fcd4d4'  # Green for buy, Red for sell
        label_color = 'limegreen' if signal in buy_signals else 'pink'

        # Create label
        label = widgets.HTML(
            value=f"<b><span style='color:{label_color};'>{signal} ATR×</span></b>",
            layout=widgets.Layout(width='auto'),
        )

        # Create slider
        multiplier = widgets.FloatSlider(
            value=atr_settings.get(f'{signal}_multiplier', 1.8),
            min=0.5,
            max=5.0,
            step=0.1,
            description='',
            readout_format='.1f',
            layout=widgets.Layout(width='90%')
        )

        # Store reference
        atr_multiplier_inputs[signal] = multiplier

        # Create container
        vbox = widgets.VBox(
            [label, multiplier],
            layout=widgets.Layout(
                margin='5px 10px 5px 10px',
                padding='10px',
                border='2px solid',
                border_color='gray',
                border_radius='5px',
                background_color=theme
            )
        )

        # Add to grid
        multiplier_grid[row, col] = vbox

    # Add all elements to the ATR container
    atr_container.children = [
        widgets.HTML(value="<h4>ATR Configuration</h4>"),
        use_atr_checkbox,
        atr_period_slider,
        widgets.HTML(value="<h4>Trailing Stop Configuration</h4>"),
        use_trailing_stop_checkbox,
        activation_slider
    ]

    # Display ATR settings
    display(atr_container)

    # Display multiplier grid
    atr_multiplier_container.children = [multiplier_grid]
    display(atr_multiplier_container)

    # Link checkbox states
    def on_use_atr_changed(change):
        if change['name'] == 'value':
            atr_period_slider.disabled = not change['new']
            use_trailing_stop_checkbox.disabled = not change['new']
            atr_multiplier_container.layout.display = 'flex' if change['new'] else 'none'

            # If ATR is disabled, also disable trailing stop
            if not change['new']:
                use_trailing_stop_checkbox.value = False

    def on_use_trailing_stop_changed(change):
        if change['name'] == 'value':
            activation_slider.disabled = not change['new']

    use_atr_checkbox.observe(on_use_atr_changed, names='value')
    use_trailing_stop_checkbox.observe(on_use_trailing_stop_changed, names='value')

    # Define grid layout: SL | RR | Signal Distance

    grid_sl = GridspecLayout(num_rows, num_columns, width='100%')
    grid_rr = GridspecLayout(num_rows, num_columns, width='100%')  # RR grid placed next to SL
    grid_sd = GridspecLayout(num_rows, num_columns, width='100%')  # Signal Distance grid placed below SL and RR

    # Function to create widgets for Stop-Loss, RR, and Signal Distance
    def create_signal_widgets(signal, current_sl=0.0, current_rr=2.0, current_sd=100):
        # Determine if the signal is a buy or sell signal
        if signal in buy_signals:
            theme_sl = '#d4fcd4'  # Light lime green background
            label_color_sl = 'limegreen'
            theme_rr = '#d4fcd4'
            label_color_rr = 'limegreen'
            theme_sd = '#d4fcd4'
            label_color_sd = 'limegreen'
        else:
            theme_sl = '#fcd4d4'  # Light pink background
            label_color_sl = 'pink'
            theme_rr = '#fcd4d4'
            label_color_rr = 'pink'
            theme_sd = '#fcd4d4'
            label_color_sd = 'pink'

        # Stop-Loss Widgets (FloatText only, step=0.01)
        label_sl = widgets.HTML(
            value=f"<b><span style='color:{label_color_sl};'>{signal} SL (%)</span></b>",
            layout=widgets.Layout(width='auto'),
        )

        float_text_sl = widgets.FloatText(
            value=sl_adjustments.get(signal, current_sl),
            disabled=False,
            step=0.01,
            layout=widgets.Layout(width='100px')  # Fixed width for better alignment
        )

        signal_vbox_sl = widgets.VBox(
            [label_sl, float_text_sl],
            layout=widgets.Layout(
                margin='5px 10px 5px 10px',
                padding='10px',
                border='2px solid',
                border_color='gray',
                border_radius='5px',
                background_color=theme_sl
            )
        )

        # RR Widgets (FloatText only, step=0.1)
        label_rr = widgets.HTML(
            value=f"<b><span style='color:{label_color_rr};'>{signal} RR</span></b>",
            layout=widgets.Layout(width='auto'),
        )

        float_text_rr = widgets.FloatText(
            value=rr_adjustments.get(signal, current_rr),
            disabled=False,
            step=0.1,
            layout=widgets.Layout(width='100px')  # Fixed width for better alignment
        )

        signal_vbox_rr = widgets.VBox(
            [label_rr, float_text_rr],
            layout=widgets.Layout(
                margin='5px 10px 5px 10px',
                padding='10px',
                border='2px solid',
                border_color='gray',
                border_radius='5px',
                background_color=theme_rr
            )
        )

        # Signal Distance Widgets (IntText only)
        label_sd = widgets.HTML(
            value=f"<b><span style='color:{label_color_sd};'>{signal} Distance</span></b>",
            layout=widgets.Layout(width='auto'),
        )

        int_text_sd = widgets.IntText(
            value=signal_distance_adjustments.get(signal, current_sd),
            disabled=False,
            layout=widgets.Layout(width='100px')  # Fixed width for better alignment
        )

        signal_vbox_sd = widgets.VBox(
            [label_sd, int_text_sd],
            layout=widgets.Layout(
                margin='5px 10px 5px 10px',
                padding='10px',
                border='2px solid',
                border_color='gray',
                border_radius='5px',
                background_color=theme_sd
            )
        )

        return signal_vbox_sl, float_text_sl, signal_vbox_rr, float_text_rr, signal_vbox_sd, int_text_sd

    # Populate the grids
    for idx, signal in enumerate(signals):
        row = idx // num_columns
        col = idx % num_columns
        widget_sl, ft_sl, widget_rr, ft_rr, widget_sd, it_sd = create_signal_widgets(signal)
        grid_sl[row, col] = widget_sl
        grid_rr[row, col] = widget_rr
        grid_sd[row, col] = widget_sd
        inputs_sl[signal] = ft_sl
        inputs_rr[signal] = ft_rr
        inputs_sd[signal] = it_sd

    # Display grids
    print("### Set Stop-Loss Levels ###\n")
    display(grid_sl)
    print("\n### Set Risk-Reward (RR) Ratios ###\n")
    display(grid_rr)
    print("\n### Set Signal Distance ###\n")
    display(grid_sd)

    if existing_adjustments:
        action_button = widgets.Button(description='Update Profile', button_style='info')
    else:
        action_button = widgets.Button(description='Save Profile', button_style='info')
    back_button = widgets.Button(description='Back', button_style='warning')
    display(widgets.HBox([action_button, back_button]))

    def on_action_button_clicked(b):
        # Save the Stop-Loss adjustments
        profile_data_sl = {signal: round(inputs_sl[signal].value, 2) for signal in signals}
        # Save the RR adjustments
        profile_data_rr = {signal: round(inputs_rr[signal].value, 2) for signal in signals}
        # Save the Signal Distance adjustments
        profile_data_sd = {signal: inputs_sd[signal].value for signal in signals}

        # Save ATR settings
        atr_data = {
            'use_atr': use_atr_checkbox.value,
            'period': atr_period_slider.value,
            'use_trailing_stop': use_trailing_stop_checkbox.value,
            'activation_pct': activation_slider.value
        }

        # Save ATR multipliers for each signal
        for signal in atr_multiplier_inputs.keys():  # Use the keys from the dictionary instead of signals list
            atr_data[f'{signal}_multiplier'] = round(atr_multiplier_inputs[signal].value, 1)

        # Combine all profile data
        profile_data = {
            'asset': asset,
            'timeframe': timeframe,
            'sl_adjustments': profile_data_sl,
            'signal_distance_adjustments': profile_data_sd,
            'rr_adjustments': profile_data_rr,
            'atr_settings': atr_data
        }
        # Save to file
        profile_dir = 'profiles'
        if not os.path.exists(profile_dir):
            os.makedirs(profile_dir)
        profile_name = f"{asset}_{timeframe.replace(' ', '_')}.json"
        profile_path = os.path.join(profile_dir, profile_name)
        with open(profile_path, 'w') as f:
            json.dump(profile_data, f, indent=4)
        action = "updated" if existing_adjustments else "saved"
        print(f"\nProfile `{profile_path}` {action} successfully.\n")
        global sl_adjustments, signal_distance_adjustments, rr_adjustments
        sl_adjustments = profile_data_sl
        signal_distance_adjustments = profile_data_sd
        rr_adjustments = profile_data_rr
        proceed_to_file_upload()

    def on_back_button_clicked(b):
        main_menu()

    action_button.on_click(on_action_button_clicked)
    back_button.on_click(on_back_button_clicked)

# ============================
# Load Existing Profile
# ============================

def load_profile():
    clear_output()
    profile_dir = 'profiles'
    if not os.path.exists(profile_dir):
        print("No profiles found. Please create a new profile first.")
        main_menu()
        return
    profile_files = [f for f in os.listdir(profile_dir) if f.endswith('.json')]
    if not profile_files:
        print("No profiles found. Please create a new profile first.")
        main_menu()
        return
    profile_dropdown = widgets.Dropdown(options=profile_files, description='Select Profile:', disabled=False)
    load_button = widgets.Button(description='Load Profile', button_style='success')
    back_button = widgets.Button(description='Back', button_style='warning')
    display(widgets.VBox([profile_dropdown, widgets.HBox([load_button, back_button])]))

    def on_load_button_clicked(b):
        profile_name = profile_dropdown.value
        if not profile_name:
            print("Please select a profile to load.")
            return
        profile_path = os.path.join('profiles', profile_name)
        with open(profile_path, 'r') as f:
            profile_data = json.load(f)
        global sl_adjustments, signal_distance_adjustments, rr_adjustments, atr_settings, selected_asset, selected_timeframe
        sl_adjustments = profile_data['sl_adjustments']
        signal_distance_adjustments = profile_data['signal_distance_adjustments']
        rr_adjustments = profile_data['rr_adjustments']
        atr_settings = profile_data['atr_settings']
        logger.debug("Loaded ATR Settings:")
        logger.debug(f"Use ATR: {atr_settings.get('use_atr', False)}")
        logger.debug(f"ATR Period: {atr_settings.get('period', 14)}")
        logger.debug(f"Use Trailing Stop: {atr_settings.get('use_trailing_stop', False)}")
        logger.debug(f"Activation %: {atr_settings.get('activation_pct', 0.5)}")
        logger.debug("ATR Multipliers:")
        for key, value in atr_settings.items():
            if key.endswith('_multiplier'):
                signal_name = key.replace('_multiplier', '')
                logger.debug(f"  Signal: {signal_name}, Multiplier: {value}")
        selected_asset = profile_data['asset']
        selected_timeframe = profile_data['timeframe']
        print(f"\nProfile `{profile_name}` loaded successfully.\n")
        # Log loaded SL, SD, and RR Adjustments
        logger.debug("Loaded Stop-Loss Adjustments:")
        for signal, sl in sl_adjustments.items():
            logger.debug(f"Signal: {signal}, SL Percentage: {sl}%")
        logger.debug("Loaded Signal Distance Adjustments:")
        for signal, sd in signal_distance_adjustments.items():
            logger.debug(f"Signal: {signal}, Signal Distance: {sd}")
        logger.debug("Loaded RR Adjustments:")
        for signal, rr in rr_adjustments.items():
            logger.debug(f"Signal: {signal}, RR Ratio: {rr}")
        proceed_to_file_upload()

    def on_back_button_clicked(b):
        main_menu()

    load_button.on_click(on_load_button_clicked)
    back_button.on_click(on_back_button_clicked)

# ============================
# Update Existing Profile
# ============================

def update_existing_profile():
    clear_output()
    profile_dir = 'profiles'
    if not os.path.exists(profile_dir):
        print("No profiles found. Please create a new profile first.")
        main_menu()
        return
    profile_files = [f for f in os.listdir(profile_dir) if f.endswith('.json')]
    if not profile_files:
        print("No profiles found. Please create a new profile first.")
        main_menu()
        return
    profile_dropdown = widgets.Dropdown(options=profile_files, description='Select Profile:', disabled=False)
    load_button = widgets.Button(description='Load for Update', button_style='warning')
    back_button = widgets.Button(description='Back', button_style='warning')
    display(widgets.VBox([profile_dropdown, widgets.HBox([load_button, back_button])]))

    def on_load_button_clicked(b):
        profile_name = profile_dropdown.value
        if not profile_name:
            print("Please select a profile to update.")
            return
        profile_path = os.path.join(profile_dir, profile_name)
        with open(profile_path, 'r') as f:
            profile_data = json.load(f)
        global sl_adjustments, signal_distance_adjustments, rr_adjustments, selected_asset, selected_timeframe
        sl_adjustments = profile_data['sl_adjustments']
        signal_distance_adjustments = profile_data['signal_distance_adjustments']
        rr_adjustments = profile_data['rr_adjustments']
        selected_asset = profile_data['asset']
        selected_timeframe = profile_data['timeframe']
        print(f"\nProfile `{profile_name}` loaded successfully.\n")
        # Log loaded SL, SD, and RR Adjustments
        logger.debug("Loaded Stop-Loss Adjustments for Update:")
        for signal, sl in sl_adjustments.items():
            logger.debug(f"Signal: {signal}, SL Percentage: {sl}%")
        logger.debug("Loaded Signal Distance Adjustments for Update:")
        for signal, sd in signal_distance_adjustments.items():
            logger.debug(f"Signal: {signal}, Signal Distance: {sd}")
        logger.debug("Loaded RR Adjustments for Update:")
        for signal, rr in rr_adjustments.items():
            logger.debug(f"Signal: {signal}, RR Ratio: {rr}")
        adjust_sl_sd_rr(selected_asset, selected_timeframe, profile_data)

    def on_back_button_clicked(b):
        main_menu()

    load_button.on_click(on_load_button_clicked)
    back_button.on_click(on_back_button_clicked)

# ============================
# Proceed to File Upload
# ============================

def proceed_to_file_upload():
    clear_output()
    print("### Upload Your CSV Files for Processing ###\n")
    upload_button = widgets.Button(description='Upload and Process', button_style='success')
    back_button = widgets.Button(description='Back', button_style='warning')
    display(widgets.HBox([upload_button, back_button]))

    def on_upload_button_clicked(b):
        uploaded_files = files.upload()
        if not uploaded_files:
            print("No files uploaded. Please upload at least one CSV file.")
            return

        for filename in uploaded_files.keys():
            print(f"\nProcessing file: {filename}")
            process_file_with_progress(filename)
        print("\nAll files have been processed. Please check the generated CSV and log files.")

    def on_back_button_clicked(b):
        main_menu()

    upload_button.on_click(on_upload_button_clicked)
    back_button.on_click(on_back_button_clicked)

# ============================
# Reinforcment Learning Prediction
# ============================

def build_rr_prediction_model(trade_log_df, use_quantile_thresholds=False, model_save_path='models/mfe_predictor', n_splits=5):
    """
    Build a model to predict Maximum Favorable Excursion (MFE) ranges.
    Modified to handle small sample sizes better.
    """
    print("Building RR Prediction ML Model (MFE Range Classification)")

    # Make a copy of the DataFrame to avoid modifying the original
    df = trade_log_df.copy()

    # Strip whitespace from column names to avoid "not in index" errors
    df.columns = df.columns.str.strip()

    # Step 1: Compute risk per trade and derive MFE risk-reward ratio
    # Calculate Risk (R) for each trade - explicitly using absolute difference
    if 'Risk' not in df.columns:
        df['Risk'] = (df['Entry_Price'] - df['SL_Price']).abs() * df['Position_Size']

    # Filter out trades with zero risk (if any) to avoid division by zero
    df = df[df['Risk'] > 0].copy()

    # Convert MFE to R-multiples (how many times the risk)
    df['MFE_R'] = df['MFE'] / df['Risk']

    # Handle extreme outliers
    q1, q3 = df['MFE_R'].quantile([0.25, 0.75])
    iqr = q3 - q1
    upper_bound = q3 + 3 * iqr
    df['MFE_R'] = df['MFE_R'].clip(upper=upper_bound)

    # Step 2: Create MFE range categories
    if use_quantile_thresholds:
        # Quantile-based thresholds (data-driven approach)
        low_threshold = df['MFE_R'].quantile(0.33)
        high_threshold = df['MFE_R'].quantile(0.67)
        print(f"\nQuantile-based thresholds: Low < {low_threshold:.2f}R, Medium {low_threshold:.2f}R-{high_threshold:.2f}R, High > {high_threshold:.2f}R")
    else:
        # Fixed thresholds based on common trading strategy targets
        low_threshold = 1.5  # MFE below 1.5R is "Low"
        high_threshold = 2.5  # MFE above 2.5R is "High"
        print(f"\nFixed thresholds: Low < 1.5R, Medium 1.5R-2.5R, High > 2.5R")

    # Create MFE range labels (1: Low, 2: Medium, 3: High)
    def label_mfe_rr(x):
        if x < low_threshold:
            return 1  # Low MFE potential
        elif x < high_threshold:
            return 2  # Medium MFE potential
        else:
            return 3  # High MFE potential

    df['MFE_Range'] = df['MFE_R'].apply(label_mfe_rr)

    # Print range distribution
    range_counts = df['MFE_Range'].value_counts()
    print("\nMFE Range Distribution:")
    for range_val, count in range_counts.items():
        range_name = ["", "Low", "Medium", "High"][range_val]
        print(f" {range_name} MFE Potential: {count} trades ({count/len(df)*100:.1f}%)")

    # Step 3: Prepare features
    feature_columns = [
        # Price action features
        'Entry_Price', 'SL_Price', 'TP_Price',
        # Indicator values
        'RSI', 'MACD_Hist', 'Ang20', 'Ang50', 'Ang100', 'Ang200',
        # Distance metrics
        'distance_20_200', 'distance_20_100', 'distance_20_50',
        'distance_50_200', 'distance_50_100', 'distance_100_200',
        # Crossover/under metrics
        'bars_since_crossup_20_200', 'bars_since_crossunder_20_200',
        'bars_since_crossup_20_100', 'bars_since_crossunder_20_100',
        'bars_since_crossup_20_50', 'bars_since_crossunder_20_50',
        'bars_since_crossup_50_200', 'bars_since_crossunder_50_200',
        'bars_since_crossup_50_100', 'bars_since_crossunder_50_100',
        'bars_since_crossup_100_200', 'bars_since_crossunder_100_200',
        # Other features
        'VWAP', 'vwap-distance', 'Volume'
    ]

    # Filter to only include columns that exist in the DataFrame
    features = [col for col in feature_columns if col in df.columns]
    print(f"\nUsing {len(features)} features for model training")

    # Create feature matrix and target vector
    X = df[features].copy()
    y = df['MFE_Range']

    # Handle missing values
    X = X.fillna(0)

    # Add trade direction as a feature if not already included
    if 'Trade_Type' in df.columns:
        X['Trade_Type_Encoded'] = df['Trade_Type'].apply(lambda x: 1 if str(x).lower() in ['long', 'buy'] else -1)
        # Add this feature to the features list if not already there
        if 'Trade_Type_Encoded' not in features:
            features.append('Trade_Type_Encoded')

    # Time-Series Cross-Validation Setup
    if 'Entry_Time' in df.columns:
        df = df.sort_values('Entry_Time').reset_index(drop=True)
        X = X.loc[df.index]
        y = y.loc[df.index]

    # Create the cross-validation object
    tscv = TimeSeriesSplit(n_splits=min(n_splits, len(df) // 10))  # Ensure enough samples per fold

    cv_results = []
    best_model = None
    best_scaler = None
    best_score = 0

    # Train on each fold
    fold = 1
    for train_index, test_index in tscv.split(X):
        print(f"\n--- Training Fold {fold} ---")
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Print class distribution for this fold
        print(f"Train set class distribution: {y_train.value_counts().to_dict()}")
        print(f"Test set class distribution: {y_test.value_counts().to_dict()}")

        # Skip this fold if any class has too few samples
        min_class_samples = y_train.value_counts().min()
        if min_class_samples < 6:
            print(f"Skipping fold {fold}: Not enough samples for SMOTE (min_class_samples={min_class_samples})")
            fold += 1
            continue

        # Scale features using RobustScaler
        scaler = RobustScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Handle class imbalance with SMOTE - using k_neighbors=min(5, min_class_samples-1)
        # This ensures we don't ask for more neighbors than exist in the smallest class
        try:
            smote = SMOTE(random_state=42, k_neighbors=min(5, min_class_samples-1))
            X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
            print(f"SMOTE resampling successful. New shape: {X_train_resampled.shape}")
        except Exception as e:
            print(f"SMOTE failed: {e}. Using original imbalanced data.")
            X_train_resampled, y_train_resampled = X_train_scaled, y_train

        # Train model
        model = LGBMClassifier(
            objective='multiclass',
            num_class=4,  # 0-indexed, so we need 4 classes for values 1-3
            random_state=42,
            n_estimators=100,
            learning_rate=0.05,
            class_weight='balanced'
        )

        model.fit(X_train_resampled, y_train_resampled)

        # Evaluate model
        y_pred = model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Fold {fold} Accuracy: {accuracy:.4f}")

        # Store evaluation results
        cv_results.append({
            'fold': fold,
            'accuracy': accuracy,
            'model': model,
            'scaler': scaler
        })

        # Update best model if this fold is better
        if accuracy > best_score:
            best_score = accuracy
            best_model = model
            best_scaler = scaler

        fold += 1

    # Check if we successfully trained any models
    if not cv_results:
        print("ERROR: Could not train any models due to insufficient data. Please provide more training data.")
        return None

    # Overall Evaluation and Visualization
    print(f"\nAverage CV Accuracy: {np.mean([r['accuracy'] for r in cv_results]):.4f}")
    print(f"Best Fold Accuracy: {best_score:.4f}")

    # Use the best model from cross-validation
    final_model = best_model
    final_scaler = best_scaler

    # Save model and artifacts
    if model_save_path:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

        # Save model, scaler, and feature list
        joblib.dump(final_model, f"{model_save_path}_model.pkl")
        joblib.dump(final_scaler, f"{model_save_path}_scaler.pkl")
        joblib.dump(features, f"{model_save_path}_features.pkl")

        # Save MFE thresholds for later use
        mfe_range_info = {
            'low_threshold': low_threshold,
            'high_threshold': high_threshold
        }
        joblib.dump(mfe_range_info, f"{model_save_path}_thresholds.pkl")

        print(f"\nModel and artifacts saved to {model_save_path}")

    # Return model and related artifacts
    return {
        'model': final_model,
        'scaler': final_scaler,
        'features': features,
        'thresholds': {
            'low_threshold': low_threshold,
            'high_threshold': high_threshold
        }
    }
# ============================
# File Processing Functions
# ============================

def process_file_with_progress(filename):
    try:
        print(f"Processing file: {filename}")
        result = process_file(filename)
        if result is None:
            print("No trade log was generated. Check the log files for errors.")
        else:
            print(f"Generated trade log with {len(result)} trades")
    except Exception as e:
        import traceback
        print(f"An error occurred while processing {filename}: {e}")
        traceback.print_exc()

def process_file(filename):

    global sl_adjustments, signal_distance_adjustments, rr_adjustments
    base_filename = os.path.splitext(filename)[0]

    # Configure a separate logger for this file
    file_logger = logging.getLogger(f'logger_{base_filename}')
    file_logger.setLevel(logging.DEBUG)

    if file_logger.hasHandlers():
        file_logger.handlers.clear()

    c_handler = logging.StreamHandler(sys.stdout)
    c_handler.setLevel(logging.ERROR)
    f_handler = logging.FileHandler(f'trading_strategy_{base_filename}.log', mode='w')
    f_handler.setLevel(logging.DEBUG)

    c_format = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
    f_format = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
    c_handler.setFormatter(c_format)
    f_handler.setFormatter(f_format)

    file_logger.addHandler(c_handler)
    file_logger.addHandler(f_handler)
    file_logger.propagate = False

    file_logger.info("================================")
    file_logger.info(f"Processing file: {filename}")
    file_logger.info("================================")

    # Log current SL, SD, and RR Adjustments
    file_logger.debug("Current Stop-Loss Adjustments:")
    for signal, sl in sl_adjustments.items():
        file_logger.debug(f"Signal: {signal}, SL Percentage: {sl}%")

    file_logger.debug("Current Signal Distance Adjustments:")
    for signal, sd in signal_distance_adjustments.items():
        file_logger.debug(f"Signal: {signal}, Signal Distance: {sd}")

    file_logger.debug("Current RR Adjustments:")
    for signal, rr in rr_adjustments.items():
        file_logger.debug(f"Signal: {signal}, RR Ratio: {rr}")

      # Check for model existence and load if available
    mfe_model = None
    if check_model_exists():
        mfe_model = load_mfe_model()
        if mfe_model:
            file_logger.info("MFE prediction model loaded successfully")
        else:
            file_logger.warning("Failed to load MFE model, using fixed TP")
    else:
        file_logger.info("No MFE model found, using fixed TP ratios")

    # Load CSV
    try:
        # Check if the file is an archive
        if filename.lower().endswith(('.zip', '.rar')):
            with open(filename, 'rb') as f:
                file_content = f.read()
            extracted = extract_csv_from_archive(file_content, filename)
            if not extracted:
                file_logger.error(f"No CSV files found in the archive {filename}.")
                return
            # For simplicity, process the first extracted CSV
            extracted_filename, df = extracted[0]
            data = df.copy()
            file_logger.info(f"Extracted and loaded CSV file: {extracted_filename}")
        else:
            data = pd.read_csv(filename)
            file_logger.info(f"CSV file '{filename}' loaded successfully.")
    except FileNotFoundError:
        file_logger.error(f"The specified CSV file '{filename}' was not found.")
        return
    except Exception as e:
        file_logger.error(f"An error occurred while loading the CSV file '{filename}': {e}")
        return

    data.columns = data.columns.str.strip()

    required_columns = ['time', 'open', 'high', 'low', 'close', 'Volume']
    missing_columns = [col for col in required_columns if col not in data.columns]
    if missing_columns:
        file_logger.error(f"The following required columns are missing from the CSV file: {missing_columns}")
        return
    else:
        file_logger.info("All required columns are present.")

    try:
        data['time'] = pd.to_datetime(data['time'], utc=True, errors='coerce')
        num_invalid = data['time'].isnull().sum()
        if num_invalid > 0:
            file_logger.warning(f"There are {num_invalid} rows with invalid 'time' format. These rows will be dropped.")
            data = data.dropna(subset=['time']).reset_index(drop=True)
        file_logger.info("'time' column parsed to datetime successfully.")
    except Exception as e:
        file_logger.error(f"An error occurred while parsing 'time' column: {e}")
        return

    from IPython.display import display
    total_rows = len(data)
    row_progress = widgets.IntProgress(value=0, min=0, max=total_rows, description='Processing:', bar_style='info')
    row_progress.layout.width = '100%'
    display(row_progress)

    utc = pytz.UTC
    berlin = pytz.timezone("Europe/Berlin")

    try:
        data['time'] = data['time'].dt.tz_convert(berlin)
        file_logger.info("'time' column converted to Europe/Berlin timezone.")
    except Exception as e:
        file_logger.error(f"An error occurred while converting 'time' to Europe/Berlin timezone: {e}")
        return

    try:
        data['Hour'] = data['time'].dt.strftime('%H:%M')
        file_logger.info("'Hour' column formatted as HH:MM.")
    except Exception as e:
        file_logger.error(f"An error occurred while formatting 'Hour' column: {e}")
        return

    def determine_session(row):
        current_hour = row['time'].hour
        if 0 <= current_hour < 8:
            return "Asia"
        elif 8 <= current_hour < 17:
            return "London"
        elif 17 <= current_hour < 24:
            return "USA"
        else:
            return "Unknown"

    try:
        data['Session'] = data.apply(determine_session, axis=1)
        file_logger.info("Session column updated.")
    except Exception as e:
        file_logger.error(f"An error occurred while determining Session column: {e}")
        return

    data.reset_index(drop=True, inplace=True)
    file_logger.info("Index reset to ensure integer indexing.")

    # Technical indicators
    N = 5
    angle_scaling_factor = 10000

    try:
        data['EMA8'] = EMAIndicator(close=data['close'], window=8).ema_indicator()
        data['EMA20'] = EMAIndicator(close=data['close'], window=20).ema_indicator()
        data['EMA50'] = EMAIndicator(close=data['close'], window=50).ema_indicator()
        data['EMA100'] = EMAIndicator(close=data['close'], window=100).ema_indicator()
        data['EMA200'] = EMAIndicator(close=data['close'], window=200).ema_indicator()
        file_logger.info("EMAs calculated.")

        data['RSI'] = RSIIndicator(close=data['close'], window=14).rsi()
        num_rsi_nan = data['RSI'].isna().sum()
        if num_rsi_nan > 0:
            data['RSI'] = data['RSI'].fillna(50)
            file_logger.warning(f"Filled {num_rsi_nan} NaN RSI values with 50.")
        data['RSI'] = data['RSI'].round(0).astype(int)
        file_logger.info("RSI calculated and rounded.")

        macd = MACD(close=data['close'], window_slow=26, window_fast=12, window_sign=9)
        data['MACD'] = macd.macd()
        data['MACD_Hist'] = macd.macd_diff()
        file_logger.info("MACD and MACD_Hist calculated.")
    except Exception as e:
        file_logger.error(f"An error occurred while calculating technical indicators: {e}")
        return

    try:
        # Calculate ATR with configurable period
        atr_period = atr_settings.get('period', 14)  # Default 14-period ATR
        data['ATR'] = AverageTrueRange(
            high=data['high'],
            low=data['low'],
            close=data['close'],
            window=atr_period
        ).average_true_range()

        # Fill any NaN values at the beginning
        data['ATR'] = data['ATR'].fillna(method='bfill')

        # Add normalized ATR (as percentage of price) for context
        data['ATR_Pct'] = (data['ATR'] / data['close'] * 100).round(2)

        file_logger.info(f"ATR calculated with period {atr_period}.")
    except Exception as e:
        file_logger.error(f"An error occurred while calculating ATR: {e}")
        return

    # Angles
    try:
        data['sufficient_bars'] = data.index >= (200 + N)

        data['delta_ema8'] = np.where(
            data['sufficient_bars'],
            data['EMA8'] - data['EMA8'].shift(N),
            np.nan
        )

        data['delta_ema20'] = np.where(
            data['sufficient_bars'],
            data['EMA20'] - data['EMA20'].shift(N),
            np.nan
        )

        data['delta_ema50'] = np.where(
            data['sufficient_bars'],
            data['EMA50'] - data['EMA50'].shift(N),
            np.nan
        )

        data['delta_ema100'] = np.where(
            data['sufficient_bars'],
            data['EMA100'] - data['EMA100'].shift(N),
            np.nan
        )

        data['delta_ema200'] = np.where(
            data['sufficient_bars'],
            data['EMA200'] - data['EMA200'].shift(N),
            np.nan
        )

        data['Ang8'] = np.degrees(np.arctan(data['delta_ema8'] / N)) * angle_scaling_factor
        data['Ang8'] = data['Ang8'].round(2)

        data['Ang20'] = np.degrees(np.arctan(data['delta_ema20'] / N)) * angle_scaling_factor
        data['Ang20'] = data['Ang20'].round(2)

        data['Ang50'] = np.degrees(np.arctan(data['delta_ema50'] / N)) * angle_scaling_factor
        data['Ang50'] = data['Ang50'].round(2)

        data['Ang100'] = np.degrees(np.arctan(data['delta_ema100'] / N)) * angle_scaling_factor
        data['Ang100'] = data['Ang100'].round(2)

        data['Ang200'] = np.degrees(np.arctan(data['delta_ema200'] / N)) * angle_scaling_factor
        data['Ang200'] = data['Ang200'].round(2)

        file_logger.info("Angles calculated.")
    except Exception as e:
        file_logger.error(f"An error occurred while calculating angles of EMAs: {e}")
        return

    # Distances
    try:
        data['distance_20_200'] = np.where(
            (~data['EMA20'].isna()) & (~data['EMA200'].isna()),
            ((data['EMA20'] - data['EMA200']) / data['EMA200']) * 100,
            np.nan
        ).round(2)

        data['distance_20_100'] = np.where(
            (~data['EMA20'].isna()) & (~data['EMA100'].isna()),
            ((data['EMA20'] - data['EMA100']) / data['EMA100']) * 100,
            np.nan
        ).round(2)

        data['distance_20_50'] = np.where(
            (~data['EMA20'].isna()) & (~data['EMA50'].isna()),
            ((data['EMA20'] - data['EMA50']) / data['EMA50']) * 100,
            np.nan
        ).round(2)

        data['distance_50_200'] = np.where(
            (~data['EMA50'].isna()) & (~data['EMA200'].isna()),
            ((data['EMA50'] - data['EMA200']) / data['EMA200']) * 100,
            np.nan
        ).round(2)

        data['distance_50_100'] = np.where(
            (~data['EMA50'].isna()) & (~data['EMA100'].isna()),
            ((data['EMA50'] - data['EMA100']) / data['EMA100']) * 100,
            np.nan
        ).round(2)

        data['distance_100_200'] = np.where(
            (~data['EMA100'].isna()) & (~data['EMA200'].isna()),
            ((data['EMA100'] - data['EMA200']) / data['EMA200']) * 100,
            np.nan
        ).round(2)

        file_logger.info("Distances between EMAs calculated.")
    except Exception as e:
        file_logger.error(f"An error occurred while calculating distances between EMAs: {e}")
        return

    # VWAP
    try:
        data['cum_volume'] = data['Volume'].cumsum()
        data['cum_volume_price'] = (data['Volume'] * data['close']).cumsum()
        data['VWAP'] = data['cum_volume_price'] / data['cum_volume']
        data['VWAP'] = data['VWAP'].round(4)
        file_logger.info("VWAP calculated.")
    except Exception as e:
        file_logger.error(f"An error occurred while calculating VWAP: {e}")
        return

    # Wick functions
    def has_upper_wick(row):
        return (row['high'] > row['close']) and \
               ((row['high'] - max(row['open'], row['close'])) > (min(row['open'], row['close']) - row['low']))

    def has_lower_wick(row):
        return (row['low'] < row['close']) and \
               ((min(row['open'], row['close']) - row['low']) > (row['high'] - max(row['open'], row['close'])))

    def has_little_upper_wick(row):
        upper_wick = row['high'] - max(row['open'], row['close'])
        total_wick = row['high'] - row['low']
        return upper_wick < (total_wick * 0.3)

    def has_little_lower_wick(row):
        lower_wick = min(row['open'], row['close']) - row['low']
        total_wick = row['high'] - row['low']
        return lower_wick < (total_wick * 0.3)

    def determine_bar_color(row):
        if row['descending_hierarchy']:
            if row['close'] < row['EMA100']:
                return '#dc01e4'
            elif row['close'] < row['EMA50']:
                return '#fab704'
            elif row['close'] < row['EMA20']:
                return '#dcfa04'
            else:
                return '#07683a'
        elif row['ascending_hierarchy']:
            if row['close'] > row['EMA100']:
                return '#6506f7'
            elif row['close'] > row['EMA50']:
                return '#06f3f7'
            elif row['close'] > row['EMA20']:
                return '#ebc904'
            else:
                return '#f70623'
        else:
            if row['close'] > row['open']:
                return '#e4ecd8'  # Light Green for Up Candle
            elif row['close'] < row['open']:
                return '#eac1b6'  # Light Red for Down Candle
            else:
                return 'rgba(0,0,0,0)'  # No color for Doji

    # Hierarchies
    try:
        data['descending_hierarchy'] = (data['EMA20'] > data['EMA50']) & \
                                       (data['EMA50'] > data['EMA100']) & \
                                       (data['EMA100'] > data['EMA200'])

        data['ascending_hierarchy'] = (data['EMA20'] < data['EMA50']) & \
                                      (data['EMA50'] < data['EMA100']) & \
                                      (data['EMA100'] < data['EMA200'])
        file_logger.info("Hierarchies defined.")
    except Exception as e:
        file_logger.error(f"An error occurred while defining hierarchies: {e}")
        return

    # Bar color
    try:
        data['bar_color'] = data.apply(determine_bar_color, axis=1)
        file_logger.info("Bar color logic applied.")
    except Exception as e:
        file_logger.error(f"An error occurred while applying bar color logic: {e}")
        return

    # Initialize signals
    signal_columns = [
        'sell20', 'sell50', 'sell200', 'continue_sell10', 'continue_sell20', 'continue_sell50',
        'buy20', 'buy50', 'buy200', 'continue_buy20', 'continue_buy50'
    ]
    for col in signal_columns:
        data[col] = False

    # Tracking Variables
    last_sell50_stoploss = np.nan
    last_sell50_signal_bar = np.nan
    last_sell20_stoploss = np.nan
    last_sell20_signal_bar = np.nan
    last_continue_sell10_stoploss = np.nan
    last_continue_sell10_signal_bar = np.nan
    last_continue_sell20_stoploss = np.nan
    last_continue_sell20_signal_bar = np.nan
    last_continue_sell50_stoploss = np.nan
    last_continue_sell50_signal_bar = np.nan

    last_buy20_stoploss = np.nan
    last_buy20_signal_bar = np.nan
    last_buy50_stoploss = np.nan
    last_buy50_signal_bar = np.nan
    last_continue_buy20_stoploss = np.nan
    last_continue_buy20_signal_bar = np.nan
    last_continue_buy50_stoploss = np.nan
    last_continue_buy50_signal_bar = np.nan

    last_buy200_stoploss = np.nan
    last_buy200_signal_bar = np.nan
    last_sell200_stoploss = np.nan
    last_sell200_signal_bar = np.nan

    cross_pairs = [
        ('20', '200'),
        ('20', '100'),
        ('20', '50'),
        ('50', '200'),
        ('50', '100'),
        ('100', '200')
    ]

    last_crossups = {f'{short}_{long}': np.nan for short, long in cross_pairs}
    last_crossunders = {f'{short}_{long}': np.nan for short, long in cross_pairs}

    # Initialize additional state variables
    gcw_found_sell = False
    yellow_search_count_sell = 0
    gcl_found_buy = False
    pink_search_count_buy = 0

    yellowish_candle_with_upper_wick_found = False
    red_candle_search_count_sell20 = 0

    light_blue_candle_with_upper_wick_found = False
    red_candle_search_count_sell50 = 0

    yellow_candle_with_lower_wick_found = False
    green_candle_search_count = 0

    orange_candle_with_lower_wick_found = False
    green_candle_search_count_buy50 = 0

    # ================================
    # === Initialize Variables for continue_sell10 Tracking ===
    # ================================
    # ================================
    last_continue_sell10_opening = np.nan  # Store the opening price of continue_sell10

    # Begin Signal Generation Loop
    for i in tqdm(range(len(data)), desc='Generating Signals'):
        row = data.iloc[i]
        if pd.isna(row['EMA8']) or pd.isna(row['EMA20']) or pd.isna(row['EMA50']) or \
           pd.isna(row['EMA100']) or pd.isna(row['EMA200']) or pd.isna(row['RSI']) or \
           pd.isna(row['MACD_Hist']):
            row_progress.value += 1
            continue

        descending_hierarchy = row['descending_hierarchy']
        ascending_hierarchy = row['ascending_hierarchy']
        current_bar_color = row['bar_color']

        if i > 0:
            prev_row = data.iloc[i-1]
            for short_ema, long_ema in cross_pairs:
                cross_key = f'{short_ema}_{long_ema}'
                short_prev = prev_row[f'EMA{short_ema}']
                long_prev = prev_row[f'EMA{long_ema}']
                short_current = row[f'EMA{short_ema}']
                long_current = row[f'EMA{long_ema}']

                if (short_prev < long_prev) and (short_current > long_current):
                    last_crossups[cross_key] = i
                if (short_prev > long_prev) and (short_current < long_current):
                    last_crossunders[cross_key] = i

        for short_ema, long_ema in cross_pairs:
            cross_key = f'{short_ema}_{long_ema}'
            last_up = last_crossups[cross_key]
            if not np.isnan(last_up):
                bars_since_up = i - last_up
                data.at[i, f'bars_since_crossup_{short_ema}_{long_ema}'] = bars_since_up
            else:
                data.at[i, f'bars_since_crossup_{short_ema}_{long_ema}'] = np.nan

            last_down = last_crossunders[cross_key]
            if not np.isnan(last_down):
                bars_since_down = i - last_down
                data.at[i, f'bars_since_crossunder_{short_ema}_{long_ema}'] = bars_since_down
            else:
                data.at[i, f'bars_since_crossunder_{short_ema}_{long_ema}'] = np.nan

        # ================================
        # === SELL20 Logic Modification ===
        # ================================
        # Sell20
        is_yellow_candle_sell = descending_hierarchy and (row['close'] < row['EMA20']) and (row['close'] > row['EMA50'])
        is_dark_green_candle = descending_hierarchy and (row['close'] > row['EMA8'])
        candle_has_upper = has_upper_wick(row)
        is_dark_green_candle_with_wick = is_dark_green_candle and candle_has_upper

        if is_dark_green_candle_with_wick:
            gcw_found_sell = True
            yellow_search_count_sell = 0

        if gcw_found_sell:
            yellow_search_count_sell += 1
            sd_threshold_sell20 = signal_distance_adjustments.get('sell20', 100)
            if yellow_search_count_sell <= sd_threshold_sell20:
                rsi_condition = row['RSI'] <= 50
                if i > 0:
                    previous_macd_hist = data.at[i-1, 'MACD_Hist']
                    macd_condition = (row['MACD_Hist'] < 0) or ((row['MACD_Hist'] > 0) and (row['MACD_Hist'] < previous_macd_hist))
                else:
                    macd_condition = False

                if is_yellow_candle_sell and rsi_condition and macd_condition:
                    # SELL20 Logic Modification with Dynamic Threshold
                    signal_name = 'sell20'
                    sd_threshold = signal_distance_adjustments.get(signal_name, 100)
                    can_generate_sell20 = False

                    if np.isnan(last_sell20_signal_bar):
                        can_generate_sell20 = True
                    elif (i - last_sell20_signal_bar) >= sd_threshold:
                        can_generate_sell20 = True
                    elif row['high'] <= last_sell20_stoploss:  # MODIFICATION: Changed >= to <=
                        can_generate_sell20 = True

                    # ================================
                    # === New Condition: Check continue_sell10 ===
                    # ================================
                    allow_generate_sell20 = True
                    if not np.isnan(last_continue_sell10_signal_bar) and (i - last_continue_sell10_signal_bar <= 50):
                        if row['high'] >= last_continue_sell10_opening:
                            allow_generate_sell20 = False  # Do not generate SELL20 signal
                    # ================================

                    if can_generate_sell20 and allow_generate_sell20:
                        data.at[i, 'sell20'] = True
                        original_sl_price = row['high']
                        adjusted_sl_price = calculate_sl_price('sell20', original_sl_price, 'sell',row)
                        data.at[i, 'SL_Price'] = adjusted_sl_price
                        # Dynamic TP calculation if model is available
                        mfe_range = None
                        if mfe_model:
                            mfe_range = predict_mfe_range(row, mfe_model)
                        # Calculate TP Price based on RR
                        tp_price = calculate_tp_price(
                            'sell20',
                            entry_price=row['close'],
                            sl_price=adjusted_sl_price,
                            signal_type='sell',
                            mfe_range=mfe_range
                        )
                        data.at[i, 'TP_Price'] = tp_price
                        last_sell20_stoploss = adjusted_sl_price
                        last_sell20_signal_bar = i
                        gcw_found_sell = False
            else:
                gcw_found_sell = False

        # ================================
        # === SELL50 Logic Modification ===
        # ================================
        # Sell50
        sell50_condition = descending_hierarchy and (row['close'] < row['EMA20']) and \
                           (row['close'] < row['EMA50']) and (row['close'] < row['EMA100']) and (row['open'] > row['EMA20'])

        # Determine if a new SELL50 signal can be generated
        sd_threshold_sell50 = signal_distance_adjustments.get('sell50', 100)
        can_generate_sell50 = True
        if not np.isnan(last_sell50_signal_bar):
            bars_since_last_sell50 = i - last_sell50_signal_bar
            if bars_since_last_sell50 < sd_threshold_sell50 and (row['high'] < last_sell50_stoploss):
                can_generate_sell50 = False  # Do not generate SELL50 signal

        if sell50_condition and can_generate_sell50:
            data.at[i, 'sell50'] = True
            original_sl_price = row['high']
            adjusted_sl_price = calculate_sl_price('sell50', original_sl_price, 'sell',row)
            data.at[i, 'SL_Price'] = adjusted_sl_price
            # Dynamic TP calculation if model is available
            mfe_range = None
            if mfe_model:
                mfe_range = predict_mfe_range(row, mfe_model)
            # Calculate TP Price based on RR
            tp_price = calculate_tp_price(
                'sell50',
                entry_price=row['close'],
                sl_price=adjusted_sl_price,
                signal_type='sell',
                mfe_range=mfe_range
            )
            data.at[i, 'TP_Price'] = tp_price
            last_sell50_stoploss = adjusted_sl_price
            last_sell50_signal_bar = i

        # ================================
        # === BUY20 Logic Modification ===
        # ================================
        # Buy20
        is_pink_candle_buy = ascending_hierarchy and (row['close'] > row['EMA20']) and (row['close'] < row['EMA50'])
        is_dark_red_candle = ascending_hierarchy and (row['close'] < row['EMA8'])
        candle_has_lower = has_lower_wick(row)
        is_dark_red_candle_with_wick = is_dark_red_candle and candle_has_lower

        if is_dark_red_candle_with_wick:
            gcl_found_buy = True
            pink_search_count_buy = 0

        if gcl_found_buy:
            pink_search_count_buy += 1
            sd_threshold_buy20 = signal_distance_adjustments.get('buy20', 100)
            if pink_search_count_buy <= sd_threshold_buy20:
                rsi_condition_buy = row['RSI'] >= 50
                if i > 0:
                    previous_macd_hist_buy = data.at[i-1, 'MACD_Hist']
                    macd_condition_buy = (row['MACD_Hist'] > 0) or ((row['MACD_Hist'] < 0) and (row['MACD_Hist'] > previous_macd_hist_buy))
                else:
                    macd_condition_buy = False

                if is_pink_candle_buy and rsi_condition_buy and macd_condition_buy:
                    # BUY20 Logic Modification with Dynamic Threshold
                    signal_name = 'buy20'
                    sd_threshold = signal_distance_adjustments.get(signal_name, 100)
                    can_generate_buy20 = False

                    if np.isnan(last_buy20_signal_bar):
                        can_generate_buy20 = True
                    elif (i - last_buy20_signal_bar) >= sd_threshold:
                        can_generate_buy20 = True
                    elif row['low'] >= last_buy20_stoploss:  # MODIFICATION: Changed <= to >=
                        can_generate_buy20 = True

                    if can_generate_buy20:
                        data.at[i, 'buy20'] = True
                        original_sl_price = row['low']
                        adjusted_sl_price = calculate_sl_price('buy20', original_sl_price, 'buy',row)
                        data.at[i, 'SL_Price'] = adjusted_sl_price
                        # Dynamic TP calculation if model is available
                        mfe_range = None
                        if mfe_model:
                            mfe_range = predict_mfe_range(row, mfe_model)
                        # Calculate TP Price based on RR
                        tp_price = calculate_tp_price(
                            'buy20',
                            entry_price=row['close'],
                            sl_price=adjusted_sl_price,
                            signal_type='buy',
                            mfe_range=mfe_range
                        )
                        data.at[i, 'TP_Price'] = tp_price
                        last_buy20_stoploss = adjusted_sl_price
                        last_buy20_signal_bar = i
                        gcl_found_buy = False
            else:
                gcl_found_buy = False

        # ================================
        # === BUY50 Logic Modification ===
        # ================================
        # Buy50
        buy50_condition = ascending_hierarchy and (row['open'] < row['EMA20']) and \
                          (row['close'] > row['EMA20']) and (row['close'] > row['EMA50']) and (row['close'] < row['EMA100'])

        # Determine if a new BUY50 signal can be generated
        sd_threshold_buy50 = signal_distance_adjustments.get('buy50', 100)
        can_generate_buy50 = True
        if not np.isnan(last_buy50_signal_bar):
            bars_since_last_buy50 = i - last_buy50_signal_bar
            if bars_since_last_buy50 < sd_threshold_buy50 and (row['low'] > last_buy50_stoploss):
                can_generate_buy50 = False  # Do not generate BUY50 signal

        if buy50_condition and can_generate_buy50:
            data.at[i, 'buy50'] = True
            original_sl_price = row['low']
            adjusted_sl_price = calculate_sl_price('buy50', original_sl_price, 'buy',row)
            data.at[i, 'SL_Price'] = adjusted_sl_price
            # Dynamic TP calculation if model is available
            mfe_range = None
            if mfe_model:
                mfe_range = predict_mfe_range(row, mfe_model)
            # Calculate TP Price based on RR
            tp_price = calculate_tp_price(
                'buy50',
                entry_price=row['close'],
                sl_price=adjusted_sl_price,
                signal_type='buy',
                mfe_range=mfe_range
            )
            data.at[i, 'TP_Price'] = tp_price
            last_buy50_stoploss = adjusted_sl_price
            last_buy50_signal_bar = i

        # ================================
        # === Continue Sell20 Signal Logic ===
        # ================================
        is_yellow_candle = ascending_hierarchy and (current_bar_color == '#ebc904')
        yellow_candle_with_upper_wick = is_yellow_candle and has_upper_wick(row)

        if yellow_candle_with_upper_wick:
            yellowish_candle_with_upper_wick_found = True
            red_candle_search_count_sell20 = 0

        if yellowish_candle_with_upper_wick_found:
            red_candle_search_count_sell20 += 1
            sd_threshold_continue_sell20 = signal_distance_adjustments.get('continue_sell20', 100)
            if red_candle_search_count_sell20 <= sd_threshold_continue_sell20:
                is_red_candle_sell20 = ascending_hierarchy and (current_bar_color == '#f70623') and (row['close'] < row['open'])
                if is_red_candle_sell20 and has_little_upper_wick(row):
                    # CONTINUE_SELL20 Logic Modification with Dynamic Threshold
                    signal_name = 'continue_sell20'
                    sd_threshold = signal_distance_adjustments.get(signal_name, 100)
                    can_generate_continue_sell20 = False

                    if np.isnan(last_continue_sell20_signal_bar):
                        can_generate_continue_sell20 = True
                    elif (i - last_continue_sell20_signal_bar) >= sd_threshold:
                        can_generate_continue_sell20 = True
                    elif row['high'] >= last_continue_sell20_stoploss:
                        can_generate_continue_sell20 = True

                    # ================================
                    # === New Condition: Check continue_sell10 ===
                    # ================================
                    allow_generate_continue_sell20 = True
                    if not np.isnan(last_continue_sell10_signal_bar) and (i - last_continue_sell10_signal_bar <= 50):
                        if row['high'] >= last_continue_sell10_opening:
                            allow_generate_continue_sell20 = False  # Do not generate Continue Sell 20 signal
                    # ================================

                    if can_generate_continue_sell20 and allow_generate_continue_sell20:
                        data.at[i, 'continue_sell20'] = True
                        original_sl_price = row['high']
                        adjusted_sl_price = calculate_sl_price('continue_sell20', original_sl_price, 'sell',row)
                        data.at[i, 'SL_Price'] = adjusted_sl_price
                        # Dynamic TP calculation if model is available
                        mfe_range = None
                        if mfe_model:
                            mfe_range = predict_mfe_range(row, mfe_model)
                        # Calculate TP Price based on RR
                        tp_price = calculate_tp_price(
                            'continue_sell20',
                            entry_price=row['close'],
                            sl_price=adjusted_sl_price,
                            signal_type='sell',
                            mfe_range=mfe_range
                        )
                        data.at[i, 'TP_Price'] = tp_price
                        last_continue_sell20_stoploss = adjusted_sl_price
                        last_continue_sell20_signal_bar = i
                        yellowish_candle_with_upper_wick_found = False
            else:
                yellowish_candle_with_upper_wick_found = False

        # ================================
        # === Continue Sell50 Signal Logic ===
        # ================================
        is_light_blue_candle_sell50 = ascending_hierarchy and (current_bar_color == '#06f3f7')
        light_blue_candle_with_upper_wick = is_light_blue_candle_sell50 and has_upper_wick(row)

        if light_blue_candle_with_upper_wick:
            light_blue_candle_with_upper_wick_found = True
            red_candle_search_count_sell50 = 0

        if light_blue_candle_with_upper_wick_found:
            red_candle_search_count_sell50 += 1
            sd_threshold_continue_sell50 = signal_distance_adjustments.get('continue_sell50', 100)
            if red_candle_search_count_sell50 <= sd_threshold_continue_sell50:
                is_red_candle_sell50 = ascending_hierarchy and (current_bar_color == '#f70623') and (row['close'] < row['open'])
                if is_red_candle_sell50 and has_little_upper_wick(row):
                    # CONTINUE_SELL50 Logic Modification with Dynamic Threshold
                    signal_name = 'continue_sell50'
                    sd_threshold = signal_distance_adjustments.get(signal_name, 100)
                    can_generate_continue_sell50 = False

                    if np.isnan(last_continue_sell50_signal_bar):
                        can_generate_continue_sell50 = True
                    elif (i - last_continue_sell50_signal_bar) >= sd_threshold:
                        can_generate_continue_sell50 = True
                    elif row['high'] <= last_continue_sell50_stoploss:  # MODIFICATION: Changed >= to <=
                        can_generate_continue_sell50 = True

                    # ================================
                    # === New Condition: Check continue_sell10 ===
                    # ================================
                    allow_generate_continue_sell50 = True
                    if not np.isnan(last_continue_sell10_signal_bar) and (i - last_continue_sell10_signal_bar <= 50):
                        if row['high'] >= last_continue_sell10_opening:
                            allow_generate_continue_sell50 = False  # Do not generate Continue Sell 50 signal
                    # ================================

                    if can_generate_continue_sell50 and allow_generate_continue_sell50:
                        data.at[i, 'continue_sell50'] = True
                        original_sl_price = row['high']
                        adjusted_sl_price = calculate_sl_price('continue_sell50', original_sl_price, 'sell',row)
                        data.at[i, 'SL_Price'] = adjusted_sl_price
                        # Dynamic TP calculation if model is available
                        mfe_range = None
                        if mfe_model:
                            mfe_range = predict_mfe_range(row, mfe_model)
                        # Calculate TP Price based on RR
                        tp_price = calculate_tp_price(
                            'continue_sell50',
                            entry_price=row['close'],
                            sl_price=adjusted_sl_price,
                            signal_type='sell',
                            mfe_range=mfe_range
                        )
                        data.at[i, 'TP_Price'] = tp_price
                        last_continue_sell50_stoploss = adjusted_sl_price
                        last_continue_sell50_signal_bar = i
                        light_blue_candle_with_upper_wick_found = False
            else:
                light_blue_candle_with_upper_wick_found = False

        # ================================
        # === Continue Sell10 Signal Logic ===
        # ================================
        # Condition from original logic: ascending_hierarchy and certain candle conditions
        is_continue_sell10_candle = ascending_hierarchy and (row['EMA8'] < row['EMA20']) and \
                                    (row['open'] > row['EMA8']) and (row['close'] < row['EMA8']) and \
                                    (current_bar_color == '#f70623')
        little_upper_wick_continue_sell10 = has_little_upper_wick(row)

        if is_continue_sell10_candle and little_upper_wick_continue_sell10:
            # Continue Sell10 Logic Modification with Dynamic Threshold
            signal_name = 'continue_sell10'
            sd_threshold_continue_sell10 = signal_distance_adjustments.get(signal_name, 100)
            can_generate_continue_sell10 = False

            if np.isnan(last_continue_sell10_signal_bar):
                can_generate_continue_sell10 = True
            elif (i - last_continue_sell10_signal_bar) >= sd_threshold_continue_sell10:
                can_generate_continue_sell10 = True
            elif row['high'] >= last_continue_sell10_stoploss:
                can_generate_continue_sell10 = True

            if can_generate_continue_sell10:
                data.at[i, 'continue_sell10'] = True
                original_sl_price = row['high']
                adjusted_sl_price = calculate_sl_price('continue_sell10', original_sl_price, 'sell',row)
                data.at[i, 'SL_Price'] = adjusted_sl_price
                # Dynamic TP calculation if model is available
                mfe_range = None
                if mfe_model:
                    mfe_range = predict_mfe_range(row, mfe_model)
                # Calculate TP Price based on RR
                tp_price = calculate_tp_price(
                    'continue_sell10',
                    entry_price=row['close'],
                    sl_price=adjusted_sl_price,
                    signal_type='sell',
                    mfe_range=mfe_range
                )
                data.at[i, 'TP_Price'] = tp_price
                last_continue_sell10_stoploss = adjusted_sl_price
                last_continue_sell10_signal_bar = i
                last_continue_sell10_opening = row['open']  # Store the Opening Price of Continue Sell10 Signal

        # ================================
        # === Continue Buy20 Signal Logic ===
        # ================================
        is_yellow_candle = descending_hierarchy and (current_bar_color == '#dcfa04')
        yellow_candle_with_lower_wick = is_yellow_candle and has_lower_wick(row)

        if yellow_candle_with_lower_wick:
            yellow_candle_with_lower_wick_found = True
            green_candle_search_count = 0

        if yellow_candle_with_lower_wick_found:
            green_candle_search_count += 1
            sd_threshold_continue_buy20 = signal_distance_adjustments.get('continue_buy20', 100)
            if green_candle_search_count <= sd_threshold_continue_buy20:
                is_green_candle_buy20 = descending_hierarchy and (current_bar_color == '#07683a') and (row['close'] > row['open'])
                if is_green_candle_buy20:
                    # Continue Buy20 Logic Modification with Dynamic Threshold
                    signal_name = 'continue_buy20'
                    sd_threshold = signal_distance_adjustments.get(signal_name, 100)
                    can_generate_continue_buy20 = False

                    if np.isnan(last_continue_buy20_signal_bar):
                        can_generate_continue_buy20 = True
                    elif (i - last_continue_buy20_signal_bar) >= sd_threshold:
                        can_generate_continue_buy20 = True
                    elif row['low'] <= last_continue_buy20_stoploss:
                        can_generate_continue_buy20 = True


                    if can_generate_continue_buy20:
                        data.at[i, 'continue_buy20'] = True
                        original_sl_price = row['low']
                        adjusted_sl_price = calculate_sl_price('continue_buy20', original_sl_price, 'buy',row)
                        data.at[i, 'SL_Price'] = adjusted_sl_price
                        # Dynamic TP calculation if model is available
                        mfe_range = None
                        if mfe_model:
                            mfe_range = predict_mfe_range(row, mfe_model)
                        # Calculate TP Price based on RR
                        tp_price = calculate_tp_price(
                            'continue_buy20',
                            entry_price=row['close'],
                            sl_price=adjusted_sl_price,
                            signal_type='buy',
                            mfe_range=mfe_range
                        )
                        last_continue_buy20_stoploss = adjusted_sl_price
                        last_continue_buy20_signal_bar = i
                        yellow_candle_with_lower_wick_found = False
            else:
                yellow_candle_with_lower_wick_found = False

        # ================================
        # === Continue Buy50 Signal Logic ===
        # ================================
        is_orange_candle = descending_hierarchy and (current_bar_color == '#fab704')
        orange_candle_with_lower_wick = is_orange_candle and has_lower_wick(row)
        close_below_50ema_with_lower_wick = descending_hierarchy and (row['close'] < row['EMA50']) and has_lower_wick(row)

        if orange_candle_with_lower_wick or close_below_50ema_with_lower_wick:
            orange_candle_with_lower_wick_found = True
            green_candle_search_count_buy50 = 0

        if orange_candle_with_lower_wick_found:
            green_candle_search_count_buy50 += 1
            sd_threshold_continue_buy50 = signal_distance_adjustments.get('continue_buy50', 100)
            if green_candle_search_count_buy50 <= sd_threshold_continue_buy50:
                is_green_candle_buy50 = descending_hierarchy and (current_bar_color == '#07683a') and (row['close'] > row['open'])
                if is_green_candle_buy50:
                    # Continue Buy50 Signal Logic Modification with Dynamic Threshold
                    signal_name = 'continue_buy50'
                    sd_threshold = signal_distance_adjustments.get(signal_name, 100)
                    can_generate_continue_buy50 = False

                    if np.isnan(last_continue_buy50_signal_bar):
                        can_generate_continue_buy50 = True
                    elif (i - last_continue_buy50_signal_bar) >= sd_threshold:
                        can_generate_continue_buy50 = True
                    elif row['low'] <= last_continue_buy50_stoploss:
                        can_generate_continue_buy50 = True

                    if can_generate_continue_buy50:
                        data.at[i, 'continue_buy50'] = True
                        original_sl_price = row['low']
                        adjusted_sl_price = calculate_sl_price('continue_buy50', original_sl_price, 'buy',row)
                        data.at[i, 'SL_Price'] = adjusted_sl_price
                        # Dynamic TP calculation if model is available
                        mfe_range = None
                        if mfe_model:
                            mfe_range = predict_mfe_range(row, mfe_model)
                        # Calculate TP Price based on RR
                        tp_price = calculate_tp_price(
                            'continue_buy50',
                            entry_price=row['close'],
                            sl_price=adjusted_sl_price,
                            signal_type='buy',
                            mfe_range=mfe_range
                        )
                        data.at[i, 'TP_Price'] = tp_price
                        last_continue_buy50_stoploss = adjusted_sl_price
                        last_continue_buy50_signal_bar = i
                        orange_candle_with_lower_wick_found = False
            else:
                orange_candle_with_lower_wick_found = False

        # ================================
        # === Buy200 Signal Logic ===
        # ================================
        is_light_green_candle = (current_bar_color == '#e4ecd8')  # Light Green for Up Candle (#e4ecd8)
        # *** MODIFICATION START ***
        # Added condition: EMA50 < EMA200 and more than 50% of the candle's body is above EMA200
        body_mid_buy200 = (row['open'] + row['close']) / 2
        body_above_200ema = body_mid_buy200 > row['EMA200']
        is_buy200_signal = is_light_green_candle and ((row['open'] < row['EMA200']) or (row['low'] < row['EMA200'])) and (row['close'] > row['EMA200']) and (row['EMA50'] < row['EMA200']) and body_above_200ema
        # *** MODIFICATION END ***

        # Determine if a new Buy200 signal can be generated
        sd_threshold_buy200 = signal_distance_adjustments.get('buy200', 100)
        can_generate_buy200 = True
        if not np.isnan(last_buy200_signal_bar):
            bars_since_last_buy200 = i - last_buy200_signal_bar
            if bars_since_last_buy200 < sd_threshold_buy200 and (row['low'] > last_buy200_stoploss):
                can_generate_buy200 = False  # Do not generate Buy200 signal

        if is_buy200_signal and can_generate_buy200:
            data.at[i, 'buy200'] = True
            original_sl_price = row['low']
            adjusted_sl_price = calculate_sl_price('buy200', original_sl_price, 'buy',row)
            # *** MODIFICATION END ***
            data.at[i, 'SL_Price'] = adjusted_sl_price
            # Dynamic TP calculation if model is available
            mfe_range = None
            if mfe_model:
                mfe_range = predict_mfe_range(row, mfe_model)
            # Calculate TP Price based on RR
            tp_price = calculate_tp_price(
                'buy200',
                entry_price=row['close'],
                sl_price=adjusted_sl_price,
                signal_type='buy',
                mfe_range=mfe_range
            )
            data.at[i, 'TP_Price'] = tp_price
            last_buy200_stoploss = adjusted_sl_price
            last_buy200_signal_bar = i

        # ================================
        # === Sell200 Signal Logic ===
        # ================================
        is_light_red_candle = (current_bar_color == '#eac1b6')  # Light Red for Down Candle (#eac1b6)
        # *** MODIFICATION START ***
        # Added condition: EMA50 > EMA200 and more than 50% of the candle's body is below EMA200
        body_mid_sell200 = (row['open'] + row['close']) / 2
        body_below_200ema = body_mid_sell200 < row['EMA200']
        is_sell200_signal = is_light_red_candle and ((row['open'] > row['EMA200']) or (row['high'] > row['EMA200'])) and (row['close'] < row['EMA200']) and (row['EMA50'] > row['EMA200']) and body_below_200ema
        # *** MODIFICATION END ***

        # Determine if a new Sell200 signal can be generated
        sd_threshold_sell200 = signal_distance_adjustments.get('sell200', 100)
        can_generate_sell200 = True
        if not np.isnan(last_sell200_signal_bar):
            bars_since_last_sell200 = i - last_sell200_signal_bar
            if bars_since_last_sell200 < sd_threshold_sell200 and (row['high'] < last_sell200_stoploss):
                can_generate_sell200 = False  # Do not generate Sell200 signal

        if is_sell200_signal and can_generate_sell200:
            data.at[i, 'sell200'] = True
            original_sl_price = row['high']
            adjusted_sl_price = calculate_sl_price('sell200', original_sl_price, 'sell',row)
            # *** MODIFICATION END ***
            data.at[i, 'SL_Price'] = adjusted_sl_price
            # Dynamic TP calculation if model is available
            mfe_range = None
            if mfe_model:
                mfe_range = predict_mfe_range(row, mfe_model)
            # Calculate TP Price based on RR
            tp_price = calculate_tp_price(
                'sell200',
                entry_price=row['close'],
                sl_price=adjusted_sl_price,
                signal_type='sell',
                mfe_range=mfe_range
            )
            data.at[i, 'TP_Price'] = tp_price
            last_sell200_stoploss = adjusted_sl_price
            last_sell200_signal_bar = i

        row_progress.value += 1

    logger.info("Trading signals generation completed.")
    row_progress.close()


    # Combine all signals
    combined_signals = pd.concat([
        data[data['sell20']].assign(signal='sell20'),
        data[data['sell50']].assign(signal='sell50'),
        data[data['sell200']].assign(signal='sell200'),
        data[data['continue_sell10']].assign(signal='continue_sell10'),
        data[data['continue_sell20']].assign(signal='continue_sell20'),
        data[data['continue_sell50']].assign(signal='continue_sell50'),
        data[data['buy20']].assign(signal='buy20'),
        data[data['buy50']].assign(signal='buy50'),
        data[data['buy200']].assign(signal='buy200'),
        data[data['continue_buy20']].assign(signal='continue_buy20'),
        data[data['continue_buy50']].assign(signal='continue_buy50'),
    ])

    # *** MODIFICATION START ***
    # Define priority order: higher priority signals come first
    priority_order = [
        'continue_sell50',
        'continue_sell20',
        'continue_sell10',
        'sell200',
        'sell50',
        'sell20',
        'continue_buy50',
        'continue_buy20',
        'buy200',
        'buy50',
        'buy20',
    ]

    # Assign priority based on signal
    combined_signals['priority'] = combined_signals['signal'].map(
        lambda x: priority_order.index(x) if x in priority_order else len(priority_order)
    )

    # Sort by time and priority
    combined_signals.sort_values(['time', 'priority'], inplace=True)

    # Drop duplicates, keeping the first (highest priority signal per time)
    combined_signals = combined_signals.drop_duplicates(subset='time', keep='first')

    # Drop the priority column
    combined_signals.drop('priority', axis=1, inplace=True)
    # *** MODIFICATION END ***

    combined_signals.sort_values('time', inplace=True)

    combined_signals['SL_Price'] = combined_signals['SL_Price'].astype(float)
    combined_signals['TP_Price'] = combined_signals['TP_Price'].astype(float)
    combined_signals['vwap-distance'] = (combined_signals['close'] - combined_signals['VWAP']).round(4)

    # Define crossup and crossunder columns
    crossup_cols = [f'bars_since_crossup_{short}_{long}' for short, long in cross_pairs]
    crossunder_cols = [f'bars_since_crossunder_{short}_{long}' for short, long in cross_pairs]

    combined_signals_export = combined_signals[['time', 'Session', 'Hour', 'signal', 'SL_Price', 'TP_Price', 'open', 'high', 'low',
                                                'close', 'Volume', 'RSI', 'MACD_Hist', 'Ang20', 'Ang50', 'Ang100', 'Ang200',
                                                'distance_20_200', 'distance_20_100', 'distance_20_50',
                                                'distance_50_200', 'distance_50_100', 'distance_100_200',
                                                *crossup_cols, *crossunder_cols,
                                                'VWAP', 'vwap-distance']].copy()

    combined_signals_export['signal'] = combined_signals_export['signal'].str.strip().str.lower()

    # Define which signals set crossup or crossunder to 0
    signals_set_crossup_to_zero = ['buy20', 'buy50', 'buy200', 'continue_sell10', 'continue_sell20', 'continue_sell50']
    signals_set_crossunder_to_zero = ['sell20', 'sell50', 'sell200', 'continue_buy20', 'continue_buy50']

    combined_signals_export.loc[combined_signals_export['signal'].isin(signals_set_crossup_to_zero), crossup_cols] = 0
    combined_signals_export.loc[combined_signals_export['signal'].isin(signals_set_crossunder_to_zero), crossunder_cols] = 0

    # Rearrange columns
    combined_signals_export = combined_signals_export[[
        'time', 'Hour', 'Session', 'signal', 'SL_Price', 'TP_Price', 'open', 'high', 'low',
        'close', 'Volume', 'RSI', 'MACD_Hist', 'Ang20', 'Ang50', 'Ang100', 'Ang200',
        'distance_20_200', 'distance_20_100', 'distance_20_50',
        'distance_50_200', 'distance_50_100', 'distance_100_200',
        'bars_since_crossup_20_200', 'bars_since_crossunder_20_200',
        'bars_since_crossup_20_100', 'bars_since_crossunder_20_100',
        'bars_since_crossup_20_50', 'bars_since_crossunder_20_50',
        'bars_since_crossup_50_200', 'bars_since_crossunder_50_200',
        'bars_since_crossup_50_100', 'bars_since_crossunder_50_100',
        'bars_since_crossup_100_200', 'bars_since_crossunder_100_200',
        'VWAP', 'vwap-distance'
    ]]

    # Handle missing SL_Price or TP_Price
    missing_sl_price = combined_signals_export['SL_Price'].isna().sum()
    missing_tp_price = combined_signals_export['TP_Price'].isna().sum()
    if missing_sl_price > 0:
        file_logger.warning(f"{missing_sl_price} signals have missing 'SL_Price' and will be excluded.")
    if missing_tp_price > 0:
        file_logger.warning(f"{missing_tp_price} signals have missing 'TP_Price' and will be excluded.")
    combined_signals_export = combined_signals_export.dropna(subset=['SL_Price', 'TP_Price'])

    # Export combined signals
    combined_signals_export.to_csv(f'combined_signals_{base_filename}.csv', index=False)
    file_logger.info(f"Combined signals exported with {len(combined_signals_export)} entries.")

    # Backtesting
    file_logger.info("Starting backtesting module.")
    file_logger.info(f"Starting backtesting with {len(combined_signals_export)} signals")
    combined_signals_export = combined_signals_export.reset_index(drop=True)
    file_logger.info(f"Reset index of combined_signals_export. Shape: {combined_signals_export.shape}")
    file_logger.debug(f"First 3 signals: {combined_signals_export[['time', 'signal', 'close', 'SL_Price', 'TP_Price']].head(3).to_dict('records')}")

    initial_capital = 100000
    risk_per_trade = 1000  # Fixed risk per trade in dollars
    pip_value = 0.0001
    balance = initial_capital
    equity = initial_capital
    max_drawdown = 0
    peak_equity = initial_capital
    trade_log = []
        # Near the start of your backtesting loop
    # Initialize drawdown tracking
    drawdown_status = {
        'peak_equity': initial_capital,
        'daily_peak': initial_capital,
        'current_equity': initial_capital,
        'total_drawdown_pct': 0,
        'daily_drawdown_pct': 0
    }


    data.set_index('time', inplace=True)
    price_data = data[['open', 'high', 'low', 'close' , 'ATR']]

    combined_signals_export.sort_values('time', inplace=True)
    combined_signals_export = combined_signals_export.reset_index(drop=True)
    trade_id_counter = 1

    trade_progress = widgets.IntProgress(value=0, min=0, max=len(combined_signals_export),
                                  description='Backtesting:', bar_style='info')
    trade_progress.layout.width = '100%'
    display(trade_progress)

    signal_count = 0
    for idx, signal_row in combined_signals_export.iterrows():
        signal_count += 1
        profit = 0
        try:
            file_logger.debug(f"Processing signal {signal_count}/{len(combined_signals_export)}: {signal_row['signal']} at {signal_row['time']}")
            signal_time = signal_row['time']
            signal_type = signal_row['signal']
            entry_price = signal_row['close']

            trade_direction = 'long' if signal_type in ['buy20', 'buy50', 'buy200', 'continue_buy20', 'continue_buy50'] else 'short'

            sl_price = signal_row['SL_Price']
            tp_price = signal_row['TP_Price']

            # Calculate SL distance in pips
            if trade_direction == 'long':
                sl_distance = entry_price - sl_price
                tp_distance = tp_price - entry_price
            else:
                sl_distance = sl_price - entry_price
                tp_distance = entry_price - tp_price

            distance_pips = sl_distance / pip_value

            # Calculate position size based on fixed risk per trade
            # Get the checkbox value
            use_dynamic_position_sizing = False
            try:
                # Access the checkbox that you can now see in the UI
                if 'dynamic_ps_checkbox' in globals():
                    use_dynamic_position_sizing = dynamic_ps_checkbox.value
            except Exception as e:
                file_logger.error(f"Error accessing dynamic position sizing checkbox: {e}")

            # Get Win/Loss model path from the text input
            wl_model_path = "models/win_loss_predictor"  # Default
            try:
                wl_model_path = wl_model_path_text.value
            except:
                pass

            if use_dynamic_position_sizing:
                # Try to load Win/Loss model
                if 'win_loss_model' not in locals() or win_loss_model is None:
                    win_loss_model = load_wl_model(wl_model_path)

                if win_loss_model:
                    # Calculate dynamic position size
                    win_probability = predict_win_probability(signal_row, win_loss_model)

                    # Calculate position size in dollars
                    dollar_position = calculate_dynamic_position_size(
                        win_probability,
                        equity,
                        fixed_predicted_rr=1.5,
                        base_kelly_fraction=0.2,
                        max_position_size_percentage=0.05
                    )

                    # Convert to position size based on sl_distance
                    position_size = dollar_position / sl_distance if sl_distance != 0 else 0

                    file_logger.info(f"Dynamic sizing: Win prob={win_probability:.4f}, Size={position_size:.2f}")
                else:
                   # Risk 1% of balance on each trade
                    risk_amount = balance * 0.01  # $1000 for $100k account

                    # Calculate position size based on entry/SL distance
                    if sl_distance > 0:
                        # Standard position sizing formula
                        pip_size = 0.0001  # For 4-decimal forex pairs
                        pip_distance = sl_distance / pip_size
                        pip_value = 10  # $10 per pip for standard lot (100k units)

                        # Calculate position size in units
                        position_size = (risk_amount / (pip_distance * pip_value)) * 100000

                        file_logger.debug(f"POSITION: Entry={entry_price}, SL={sl_price}, Distance={sl_distance:.5f}")
                        file_logger.debug(f"Pip distance={pip_distance:.1f}, Risk=${risk_amount}, Position={position_size:.0f}")
                    else:
                        position_size = 0  # Safety check
            else:
                # Risk 1% of balance on each trade
                    risk_amount = balance * 0.01  # $1000 for $100k account

                    # Calculate position size based on entry/SL distance
                    if sl_distance > 0:
                        # Standard position sizing formula
                        pip_size = 0.0001  # For 4-decimal forex pairs
                        pip_distance = sl_distance / pip_size
                        pip_value = 10  # $10 per pip for standard lot (100k units)

                        # Calculate position size in units
                        position_size = (risk_amount / (pip_distance * pip_value)) * 100000

                        file_logger.debug(f"POSITION: Entry={entry_price}, SL={sl_price}, Distance={sl_distance:.5f}")
                        file_logger.debug(f"Pip distance={pip_distance:.1f}, Risk=${risk_amount}, Position={position_size:.0f}")
                    else:
                        position_size = 0  # Safety check

            trade = {
                'Trade_ID': trade_id_counter,
                'Entry_Time': signal_time,
                'Exit_Time': None,
                'Trade_Duration': None,
                'Signal': signal_type,
                'Trade_Type': trade_direction,
                'Entry_Price': entry_price,
                'Initial_SL_Price': sl_price,
                'Final_SL_Price': None,
                'Trailing_Activated': False,
                'SL_Price': sl_price,
                'TP_Price': tp_price,
                'Exit_Price': None,
                'Trade_Result': None,
                'Profit': 0,
                'Position_Size': position_size,
                'MFE': 0,
                'MAE': 0,
                'RSI': signal_row['RSI'],
                'MACD_Hist': signal_row['MACD_Hist'],
                'Ang20': signal_row['Ang20'],
                'Ang50': signal_row['Ang50'],
                'Ang100': signal_row['Ang100'],
                'Ang200': signal_row['Ang200'],
                'distance_20_200': signal_row['distance_20_200'],
                'distance_20_100': signal_row['distance_20_100'],
                'distance_20_50': signal_row['distance_20_50'],
                'distance_50_200': signal_row['distance_50_200'],
                'distance_50_100': signal_row['distance_50_100'],
                'distance_100_200': signal_row['distance_100_200'],
                'bars_since_crossup_20_200': signal_row['bars_since_crossup_20_200'],
                'bars_since_crossunder_20_200': signal_row['bars_since_crossunder_20_200'],
                'bars_since_crossup_20_100': signal_row['bars_since_crossup_20_100'],
                'bars_since_crossunder_20_100': signal_row['bars_since_crossunder_20_100'],
                'bars_since_crossup_20_50': signal_row['bars_since_crossup_20_50'],
                'bars_since_crossunder_20_50': signal_row['bars_since_crossunder_20_50'],
                'bars_since_crossup_50_200': signal_row['bars_since_crossup_50_200'],
                'bars_since_crossunder_50_200': signal_row['bars_since_crossunder_50_200'],
                'bars_since_crossup_50_100': signal_row['bars_since_crossup_50_100'],
                'bars_since_crossunder_50_100': signal_row['bars_since_crossunder_50_100'],
                'bars_since_crossup_100_200': signal_row['bars_since_crossup_100_200'],
                'bars_since_crossunder_100_200': signal_row['bars_since_crossunder_100_200'],
                'VWAP': signal_row['VWAP'],
                'vwap-distance': signal_row['vwap-distance'],
                'open': signal_row['open'],
                'high': signal_row['high'],
                'low': signal_row['low'],
                'close': signal_row['close'],
                'Volume': signal_row['Volume'],
                'Session': signal_row['Session'],
                'Hour': signal_row['Hour']
            }

            trade_id_counter += 1

            entry_time = trade['Entry_Time']
            exit_time = None
            exit_price = None
            trade_result = None
            profit = 0

            # Initialize trailing stop variables
            initial_sl_price = trade['SL_Price']
            current_sl_price = initial_sl_price
            trailing_stop_activated = False
            use_trailing_stop = atr_settings.get('use_trailing_stop', False)
            trailing_activation_pct = atr_settings.get('activation_pct', 0.5)  # Default 50% toward TP
            atr_multiplier = atr_settings.get(f"{signal_type}_multiplier", 1.8)

            # Add this debug statement:
            file_logger.debug(f"ATR settings for trade {trade['Trade_ID']}: use_trailing_stop={use_trailing_stop}, " +
                    f"activation_pct={trailing_activation_pct}, atr_multiplier={atr_multiplier}")

            if 'ATR' in data.columns and signal_time in data.index:
                file_logger.debug(f"Current ATR value: {data.loc[signal_time, 'ATR']}")

            # Store these values in the trade dictionary for reporting
            trade['Initial_SL_Price'] = initial_sl_price
            trade['Trailing_Activated'] = False
            trade['Final_SL_Price'] = None
            trade['ATR_Value'] = signal_row.get('ATR', None)

            # Initialize MFE/MAE tracking variables
            max_favorable_excursion = 0
            max_adverse_excursion = 0

            # Get subsequent prices for backtesting
            subsequent_prices = price_data[price_data.index > entry_time]
            file_logger.debug(f"Found {len(subsequent_prices)} price bars after entry for trade {trade['Trade_ID']}.")

            # Set exit variables with default values
            exit_time = None
            exit_price = None
            trade_result = None
            profit = 0

            # Process each price bar
            for time, prices in subsequent_prices.iterrows():
                high = prices['high']
                low = prices['low']
                close = prices['close']
                current_atr = prices.get('ATR', 0)

                file_logger.debug(f"Trade {trade['Trade_ID']}: Bar {time}, ATR: {current_atr}, " +
                              f"Close: {close}, High: {high}, Low: {low}")

                # For long positions
                if trade['Trade_Type'] == 'long':
                    # Calculate potential profit at current bar
                    profit_at_high = (high - trade['Entry_Price']) * trade['Position_Size']
                    profit_at_low = (low - trade['Entry_Price']) * trade['Position_Size']

                    # Update MFE/MAE tracking
                    max_favorable_excursion = max(max_favorable_excursion, profit_at_high)
                    max_adverse_excursion = min(max_adverse_excursion, profit_at_low)

                    # Handle trailing stop logic
                    if use_trailing_stop and current_atr > 0:
                        # Calculate progress toward take-profit
                        if trade['TP_Price'] > trade['Entry_Price']:  # Avoid division by zero
                            price_progress = (high - trade['Entry_Price']) / (trade['TP_Price'] - trade['Entry_Price'])
                            file_logger.debug(f"Trade {trade['Trade_ID']} price progress: {price_progress:.2f}, activation threshold: {trailing_activation_pct}")

                            # Activate trailing stop once price moves sufficiently toward TP
                            if price_progress >= trailing_activation_pct and not trailing_stop_activated:
                                trailing_stop_activated = True
                                trade['Trailing_Activated'] = True
                                file_logger.debug(f"Trade {trade['Trade_ID']} trailing stop activated at {time}")

                            # If trailing is activated, calculate new stop level based on current close
                            # Track highest price reached since entry (add this at the start of trade processing)
                            if 'max_high_since_entry' not in trade:
                                trade['max_high_since_entry'] = trade['Entry_Price']

                            # Update highest high (add in each price bar iteration)
                            trade['max_high_since_entry'] = max(trade['max_high_since_entry'], high)

                            # When trailing is activated
                            if trailing_stop_activated:
                                # Simple trailing calculation: close - (multiplier * ATR)
                                new_sl = close - (atr_multiplier * current_atr)

                                # FIXED BREAKEVEN LOGIC - uses highest high instead of close
                                entry_price = trade['Entry_Price']
                                max_progress = (trade['max_high_since_entry'] - entry_price) / (trade['TP_Price'] - entry_price)

                                if max_progress >= 0.25:
                                    new_sl = max(new_sl, entry_price)

                                # Only move stop-loss up (never down) for long positions
                                if new_sl > current_sl_price:
                                    current_sl_price = new_sl
                                    file_logger.debug(f"Trade {trade['Trade_ID']} trailing stop raised to {current_sl_price:.5f}")

                    # Check if stop-loss is hit
                    if low <= current_sl_price:
                        exit_price = current_sl_price
                        exit_time = time

                        # Determine if this was a trailing stop or initial stop
                        if trailing_stop_activated and current_sl_price > initial_sl_price:
                            trade_result = 'Trailing SL Hit'
                        else:
                            trade_result = 'SL Hit'

                        profit = (exit_price - trade['Entry_Price']) * trade['Position_Size']
                        file_logger.debug(f"Trade {trade['Trade_ID']} {trade_result} at {time}, profit: {profit:.2f}")
                        break

                    # Check if take-profit is hit
                    elif high >= trade['TP_Price']:
                        exit_price = trade['TP_Price']
                        exit_time = time
                        trade_result = 'TP Hit'
                        profit = (exit_price - trade['Entry_Price']) * trade['Position_Size']
                        file_logger.debug(f"Trade {trade['Trade_ID']} TP hit at {time}, profit: {profit:.2f}")
                        break

                # For short positions
                else:  # trade['Trade_Type'] == 'short'
                    # Calculate potential profit at current bar
                    profit_at_low = (trade['Entry_Price'] - low) * trade['Position_Size']
                    profit_at_high = (trade['Entry_Price'] - high) * trade['Position_Size']

                    # Update MFE/MAE tracking
                    max_favorable_excursion = max(max_favorable_excursion, profit_at_low)
                    max_adverse_excursion = min(max_adverse_excursion, profit_at_high)

                    # Handle trailing stop logic
                    if use_trailing_stop and current_atr > 0:
                        # Calculate progress toward take-profit
                        if trade['Entry_Price'] > trade['TP_Price']:  # Avoid division by zero
                            price_progress = (trade['Entry_Price'] - low) / (trade['Entry_Price'] - trade['TP_Price'])

                            # Activate trailing stop once price moves sufficiently toward TP
                            if price_progress >= trailing_activation_pct and not trailing_stop_activated:
                                trailing_stop_activated = True
                                trade['Trailing_Activated'] = True
                                file_logger.debug(f"Trade {trade['Trade_ID']} trailing stop activated at {time}")

                            # If trailing is activated, calculate new stop level based on current close
                            # Track lowest price reached since entry
                            if 'min_low_since_entry' not in trade:
                                trade['min_low_since_entry'] = trade['Entry_Price']

                            # Update lowest low
                            trade['min_low_since_entry'] = min(trade['min_low_since_entry'], low)

                            # When trailing is activated
                            if trailing_stop_activated:
                                # Simple trailing calculation: close + (multiplier * ATR)
                                new_sl = close + (atr_multiplier * current_atr)

                                # FIXED BREAKEVEN LOGIC - uses lowest low instead of close
                                entry_price = trade['Entry_Price']
                                max_progress = (entry_price - trade['min_low_since_entry']) / (entry_price - trade['TP_Price'])

                                if max_progress >= 0.25:
                                    new_sl = min(new_sl, entry_price)

                                # Only move stop-loss down (never up) for short positions
                                if new_sl < current_sl_price:
                                    current_sl_price = new_sl
                                    file_logger.debug(f"Trade {trade['Trade_ID']} trailing stop lowered to {current_sl_price:.5f}")

                    # Check if stop-loss is hit
                    if high >= current_sl_price:
                        exit_price = current_sl_price  # Use the current (possibly trailed) stop price
                        exit_time = time

                        # Report whether it was initial or trailing stop
                        if trailing_stop_activated and current_sl_price < initial_sl_price:
                            trade_result = 'Trailing SL Hit'
                        else:
                            trade_result = 'SL Hit'

                        profit = (trade['Entry_Price'] - exit_price) * trade['Position_Size']
                        file_logger.debug(f"Trade {trade['Trade_ID']} {trade_result} at {time}, profit: {profit:.2f}")
                        break

                    # Check if take-profit is hit
                    elif low <= trade['TP_Price']:
                        exit_price = trade['TP_Price']
                        exit_time = time
                        trade_result = 'TP Hit'
                        profit = (trade['Entry_Price'] - exit_price) * trade['Position_Size']
                        file_logger.debug(f"Trade {trade['Trade_ID']} TP hit at {time}, profit: {profit:.2f}")
                        break

            # Update final stop price in trade record
            trade['Final_SL_Price'] = current_sl_price

            # Handle cases where no exit condition was met during the simulation
            if exit_time is None and not subsequent_prices.empty:
                exit_time = subsequent_prices.index[-1]
                exit_price = subsequent_prices.iloc[-1]['close']
                trade_result = 'Closed'
                if trade['Trade_Type'] == 'long':
                    profit = (exit_price - trade['Entry_Price']) * trade['Position_Size']
                else:
                    profit = (trade['Entry_Price'] - exit_price) * trade['Position_Size']
            elif exit_time is None and subsequent_prices.empty:
                exit_time = trade['Entry_Time']
                exit_price = trade['Entry_Price']
                trade_result = 'Closed'
                profit = 0

            # Set all final trade fields
            trade['Exit_Time'] = exit_time
            trade['Exit_Price'] = exit_price
            trade['Trade_Result'] = trade_result
            trade['Profit'] = profit
            trade['MFE'] = max_favorable_excursion
            trade['MAE'] = max_adverse_excursion

            # Debug log for trade completion
            file_logger.debug(f"Trade {trade['Trade_ID']} completed with Profit: {profit}, MFE: {max_favorable_excursion}, MAE: {max_adverse_excursion}")

            # Add to trade log (only once!)
            trade_log.append(trade)
            file_logger.debug(f"Added trade {trade['Trade_ID']} to trade log. Current count: {len(trade_log)}")

            # Update account balance and statistics
            balance += profit
            equity = balance
            peak_equity = max(peak_equity, equity)
            drawdown = (peak_equity - equity) / peak_equity
            max_drawdown = max(max_drawdown, drawdown)
            # Add drawdown tracking HERE
            drawdown_status['current_equity'] = equity

            # Update peaks
            if equity > drawdown_status['peak_equity']:
                drawdown_status['peak_equity'] = equity

            # Reset daily peak on new day
            if idx > 0 and signal_row['time'].date() != combined_signals_export.iloc[idx-1]['time'].date():
                drawdown_status['daily_peak'] = equity
            elif equity > drawdown_status['daily_peak']:
                drawdown_status['daily_peak'] = equity

            # Calculate drawdowns
            total_drawdown = (drawdown_status['peak_equity'] - equity) / drawdown_status['peak_equity']
            daily_drawdown = (drawdown_status['daily_peak'] - equity) / drawdown_status['daily_peak']

            drawdown_status.update({
                'total_drawdown_pct': total_drawdown,
                'daily_drawdown_pct': daily_drawdown
            })
            # Update progress bar
            trade_progress.value = signal_count
            file_logger.debug(f"Completed processing signal {signal_count}/{len(combined_signals_export)}")

        except Exception as e:
            # Log any exceptions that occur during processing
            file_logger.error(f"Error processing signal {signal_count}: {str(e)}")
            import traceback
            file_logger.error(traceback.format_exc())
            # Continue with the next signal
            continue

    # ---- MOVED OUTSIDE THE LOOP ----
    # Log completion of backtesting after all signals have been processed
    file_logger.info(f"Completed backtesting. Total signals processed: {signal_count}, trades collected: {len(trade_log)}")

    # --- Begin Trade Log Creation, Analysis, and CSV Saving ---
    # Create trade log DataFrame from the collected trade_log list
    try:
        trade_log_df = pd.DataFrame(trade_log)
    except Exception as e:
        logger.error(f"Error creating trade log DataFrame: {e}")
        raise

    # If trailing stop analysis is enabled, perform analysis and export as CSV
    if atr_settings.get('use_trailing_stop', False):
        try:
            trailing_performance = analyze_trailing_stop_performance(trade_log_df)
            trailing_analysis_df = pd.DataFrame(list(trailing_performance.items()),
                                              columns=['Metric', 'Value'])
            trailing_analysis_df.to_csv(f'trailing_analysis_{base_filename}.csv', index=False)
        except Exception as e:
            logger.error(f"Error during trailing stop analysis or saving CSV: {e}")

    # Rearrange trade log columns to the desired order
    try:
        desired_columns = ['Trade_ID', 'Entry_Time', 'Exit_Time', 'Trade_Duration', 'Signal', 'Trade_Type',
                          'Entry_Price', 'Initial_SL_Price', 'Final_SL_Price', 'Trailing_Activated',
                          'SL_Price', 'TP_Price', 'Exit_Price', 'Trade_Result', 'Profit', 'Position_Size',
                          'MFE', 'MAE', 'ATR_Value', 'RSI', 'MACD_Hist', 'Ang20', 'Ang50', 'Ang100', 'Ang200',
                          'distance_20_200', 'distance_20_100', 'distance_20_50',
                          'distance_50_200', 'distance_50_100', 'distance_100_200',
                          'bars_since_crossup_20_200', 'bars_since_crossunder_20_200',
                          'bars_since_crossup_20_100', 'bars_since_crossunder_20_100',
                          'bars_since_crossup_20_50', 'bars_since_crossunder_20_50',
                          'bars_since_crossup_50_200', 'bars_since_crossunder_50_200',
                          'bars_since_crossup_50_100', 'bars_since_crossunder_50_100',
                          'bars_since_crossup_100_200', 'bars_since_crossunder_100_200',
                          'VWAP', 'vwap-distance', 'open', 'high', 'low', 'close', 'Volume', 'Session', 'Hour']
        trade_log_df = trade_log_df[desired_columns].copy()
    except Exception as e:
        logger.error(f"Error rearranging trade log columns: {e}")

    # Save the trade log DataFrame as CSV
    try:
        trade_log_df.to_csv(f'trade_log_{base_filename}.csv', index=False)
        logger.info(f"Trade log exported to 'trade_log_{base_filename}.csv'.")
    except Exception as e:
        logger.error(f"Error saving trade log CSV: {e}")

    # Optionally print summary information to the console
    print("Available columns in trade_log_df:", trade_log_df.columns.tolist())
    print("Sample data:", trade_log_df[['Entry_Price', 'SL_Price', 'MFE', 'Position_Size']].head())

    # Compute performance metrics
    try:
        total_profit = trade_log_df['Profit'].sum()
        num_tp = trade_log_df[trade_log_df['Trade_Result'] == 'TP Hit'].shape[0]
        num_sl = trade_log_df[trade_log_df['Trade_Result'] == 'SL Hit'].shape[0]
        total_trades = len(trade_log_df)
        win_rate = (num_tp / total_trades) * 100 if total_trades > 0 else 0
        average_profit = trade_log_df['Profit'].mean()
        max_drawdown_pct = max_drawdown * 100
        profit_factor = (trade_log_df[trade_log_df['Profit'] > 0]['Profit'].sum() /
                        abs(trade_log_df[trade_log_df['Profit'] < 0]['Profit'].sum())
                        if trade_log_df[trade_log_df['Profit'] < 0]['Profit'].sum() != 0 else np.nan)

        performance_metrics = {
            'Total Trades': total_trades,
            'Winning Trades': num_tp,
            'Losing Trades': num_sl,
            'Win Rate (%)': round(win_rate, 2),
            'Total Profit': round(total_profit, 2),
            'Average Profit per Trade': round(average_profit, 2),
            'Maximum Drawdown (%)': round(max_drawdown_pct, 2),
            'Profit Factor': round(profit_factor, 2) if not np.isnan(profit_factor) else 'N/A',
            'Ending Balance': round(balance, 2)
        }

        performance_df = pd.DataFrame(list(performance_metrics.items()), columns=['Metric', 'Value'])
        performance_df.to_csv(f'performance_metrics_{base_filename}.csv', index=False)
        logger.info(f"Performance metrics exported to 'performance_metrics_{base_filename}.csv'.")
    except Exception as e:
        logger.error(f"Error computing or saving performance metrics: {e}")

    logger.info("================================")
    logger.info(f"Finished processing file: {filename}")
    logger.info("================================\n")

    # --- End Trade Log Creation, Analysis, and CSV Saving ---

    # Finally, return the trade log DataFrame if needed (MOVED TO END OF FUNCTION)
    return trade_log_df


def visualize_trailing_stop_performance(trade_log_df, base_filename):
    """
    Create and save visualizations for trailing stop performance analysis.

    Parameters:
    trade_log_df (pd.DataFrame): The trade log DataFrame
    base_filename (str): Base filename for saving charts
    """
    if not atr_settings.get('use_trailing_stop', False):
        return

    # Check if matplotlib is available
    try:
        import matplotlib.pyplot as plt
        import seaborn as sns
    except ImportError:
        logger.warning("Matplotlib or Seaborn not available. Skipping visualizations.")
        return

    # Set some styling
    plt.style.use('ggplot')

    # 1. Profit comparison: Trailing SL vs Normal SL vs TP
    plt.figure(figsize=(10, 6))
    result_profit = trade_log_df.groupby('Trade_Result')['Profit'].sum()

    colors = ['#3498db', '#e74c3c', '#2ecc71']
    ax = result_profit.plot(kind='bar', color=colors)

    plt.title('Profit Distribution by Exit Type', fontsize=14)
    plt.ylabel('Total Profit ($)', fontsize=12)
    plt.xlabel('Exit Type', fontsize=12)
    plt.xticks(rotation=45)

    # Add profit values on top of bars
    for i, p in enumerate(ax.patches):
        ax.annotate(f'${p.get_height():.2f}',
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center',
                    xytext=(0, 10), textcoords='offset points')

    plt.tight_layout()
    plt.savefig(f'trailing_profit_distribution_{base_filename}.png')

    # 2. Trade count comparison
    plt.figure(figsize=(10, 6))
    result_count = trade_log_df['Trade_Result'].value_counts()

    ax = result_count.plot(kind='pie', autopct='%1.1f%%', colors=colors)
    plt.title('Trade Outcome Distribution', fontsize=14)
    plt.ylabel('')

    plt.tight_layout()
    plt.savefig(f'trailing_outcome_distribution_{base_filename}.png')

    # 3. Profit per trade by exit type
    plt.figure(figsize=(10, 6))
    avg_profit = trade_log_df.groupby('Trade_Result')['Profit'].mean()

    ax = avg_profit.plot(kind='bar', color=colors)

    plt.title('Average Profit per Trade by Exit Type', fontsize=14)
    plt.ylabel('Average Profit ($)', fontsize=12)
    plt.xlabel('Exit Type', fontsize=12)
    plt.xticks(rotation=45)

    # Add profit values on top of bars
    for i, p in enumerate(ax.patches):
        ax.annotate(f'${p.get_height():.2f}',
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center',
                    xytext=(0, 10), textcoords='offset points')

    plt.tight_layout()
    plt.savefig(f'trailing_avg_profit_{base_filename}.png')

    # 4. Trailing stop distance analysis
    trailing_trades = trade_log_df[trade_log_df['Trailing_Activated']]

    if len(trailing_trades) > 0:
        plt.figure(figsize=(10, 6))

        # Calculate SL movement
        trailing_trades['SL_Movement'] = trailing_trades.apply(
            lambda x: (x['Final_SL_Price'] - x['Initial_SL_Price']) if x['Trade_Type'] == 'long'
                      else (x['Initial_SL_Price'] - x['Final_SL_Price']),
            axis=1
        )

        # Calculate SL movement as percentage of ATR
        trailing_trades['SL_Movement_ATR'] = trailing_trades['SL_Movement'] / trailing_trades['ATR_Value']

        # Plot histogram of SL movement in ATR units
        sns.histplot(trailing_trades['SL_Movement_ATR'], bins=20, kde=True)

        plt.title('Trailing Stop Movement (in ATR units)', fontsize=14)
        plt.xlabel('Stop-Loss Movement (× ATR)', fontsize=12)
        plt.ylabel('Frequency', fontsize=12)

        plt.tight_layout()
        plt.savefig(f'trailing_movement_{base_filename}.png')

    logger.info(f"Trailing stop visualizations saved with base name '{base_filename}'")



def comprehensive_backtest(historical_data, start_date=None, end_date=None):
    """
    Comprehensive backtesting with multiple configurations and walk-forward validation

    Args:
        historical_data: DataFrame with price data
        start_date: Start date for backtesting (None = use all data)
        end_date: End date for backtesting (None = use all data)

    Returns:
        Dictionary with backtesting results
    """
    # Filter data by date range if specified
    if start_date or end_date:
        mask = pd.Series(True, index=historical_data.index)
        if start_date:
            mask &= (historical_data.index >= pd.Timestamp(start_date))
        if end_date:
            mask &= (historical_data.index <= pd.Timestamp(end_date))
        data = historical_data[mask].copy()
    else:
        data = historical_data.copy()

    print(f"Running comprehensive backtest on {len(data)} bars from "
          f"{data.index[0]} to {data.index[-1]}")

    # Define strategy configurations to test
    strategies = {
        'Baseline': {
            'use_trailing_stop': False,
            'use_dynamic_tp': False,
            'use_dynamic_position_sizing': False
        },
        'Trailing SL Only': {
            'use_trailing_stop': True,
            'use_dynamic_tp': False,
            'use_dynamic_position_sizing': False
        },
        'Dynamic TP Only': {
            'use_trailing_stop': False,
            'use_dynamic_tp': True,
            'use_dynamic_position_sizing': False
        },
        'Dynamic Position Only': {
            'use_trailing_stop': False,
            'use_dynamic_tp': False,
            'use_dynamic_position_sizing': True
        },
        'Full Hybrid System': {
            'use_trailing_stop': True,
            'use_dynamic_tp': True,
            'use_dynamic_position_sizing': True
        }
    }

    # Run full-period backtest for each strategy
    full_period_results = {}
    for name, config in strategies.items():
        print(f"\nTesting strategy: {name}")
        trade_log = backtest_with_settings(data, **config)
        metrics = compute_performance_metrics(trade_log)

        full_period_results[name] = {
            'trade_log': trade_log,
            'metrics': metrics
        }

        # Print key metrics
        print(f"  Total Profit: ${metrics['Total Profit']:.2f}")
        print(f"  Win Rate: {metrics['Win Rate (%)']:.2f}%")
        print(f"  Profit Factor: {metrics['Profit Factor']:.2f}")
        print(f"  Max Drawdown: {metrics['Max Drawdown (%)']:.2f}%")
        print(f"  Sharpe Ratio: {metrics['Sharpe Ratio']:.2f}")

    # Visualize equity curves
    plot_equity_curves(full_period_results)

    # Run walk-forward validation for best strategy
    best_strategy = max(full_period_results.keys(),
                       key=lambda s: full_period_results[s]['metrics']['Sharpe Ratio'])

    print(f"\nRunning walk-forward validation for best strategy: {best_strategy}")
    wf_results = walk_forward_validation(data, strategies[best_strategy],
                                       window_size=180, step_size=30)

    # Run stress tests on volatile periods
    print("\nRunning stress tests on high volatility periods")
    stress_results = run_stress_tests(data, strategies['Full Hybrid System'])

    # Return comprehensive results
    return {
        'full_period': full_period_results,
        'walk_forward': wf_results,
        'stress_tests': stress_results,
        'best_strategy': best_strategy
    }

# ============================
# Start the main menu
# ============================

main_menu()


HTML(value="<h1 style='color: #2e86c1; text-align: center;'>Advanced Trading Bot</h1><h3 style='color: #138d75…

GridspecLayout(children=(VBox(children=(Button(button_style='primary', description='Generate Signal & Backtest…